# Ordering by metrics and retraining phase

## Dataset: Intel

## Experiment configuration 1


## Configuration 2
	2. Incremental guided retraining starting from the original model using the new adversarial inputs and original training set.

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
pwd

'C:\\Users\\fjdurlop\\Documents\\upc\\upc-july\\notebooks'

In [4]:
cd C:/Users/fjdurlop/Documents/upc/utils/

C:\Users\fjdurlop\Documents\upc\utils


In [5]:
# utils for project
import utils_guided_retraining2 as utils

2.5.0
keras
2.5.0


In [6]:
#cd '../notebooks/'

In [7]:
dataset = "intel"

In [8]:
x_train,y_train = utils.get_data(dataset,"Train",verbose=True)
x_val,y_val = utils.get_data(dataset,"Val",verbose=True)
x_test,y_test = utils.get_data(dataset,"Test",verbose=True)

C:/Users/fjdurlop/Documents/upc/train_data/intel/x_train_intel_2.npy
x_set len:  11224
C:/Users/fjdurlop/Documents/upc/train_data/intel/y_train_intel_2.npy
y_set len:  11224
C:/Users/fjdurlop/Documents/upc/train_data/intel/x_val_intel_2.npy
x_set len:  2804
C:/Users/fjdurlop/Documents/upc/train_data/intel/y_val_intel_2.npy
y_set len:  2804
C:/Users/fjdurlop/Documents/upc/train_data/intel/x_test_intel_2.npy
x_set len:  3000
C:/Users/fjdurlop/Documents/upc/train_data/intel/y_test_intel_2.npy
y_set len:  3000


In [9]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary",verbose=True)

C:/Users/fjdurlop/Documents/upc/data_adversarial_july/intel/train_and_adversary.npy
x_set len:  14224
C:/Users/fjdurlop/Documents/upc/data_adversarial_july/intel/train_and_adversary_labels.npy
y_set len:  14224


In [10]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

3000
3000


In [11]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [12]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

3000
3000


## ----

In [13]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"


In [14]:
model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02"

In [15]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10
Model loaded correctly


In [16]:
print(len(x_adversary_training))

3000


In [17]:
print(x_adversary_training.shape)
len(x_adversary_training)//20

(3000, 48, 48, 1)


150

In [18]:
n_data_points = 150

In [19]:
import gc

## Loading LSA and DSA values 

In [ ]:
lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"


In [20]:
lsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/lsa_values.npy"


In [21]:
lsa_values = np.load(lsa_direction)[len(x_train):] 
print(len(lsa_values))


3000


In [22]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_adversary_training)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_adversary_training)


In [23]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  150
150
1 :
0  ->  300
300
2 :
0  ->  450
450
3 :
0  ->  600
600
4 :
0  ->  750
750
5 :
0  ->  900
900
6 :
0  ->  1050
1050
7 :
0  ->  1200
1200
8 :
0  ->  1350
1350
9 :
0  ->  1500
1500
10 :
0  ->  1650
1650
11 :
0  ->  1800
1800
12 :
0  ->  1950
1950
13 :
0  ->  2100
2100
14 :
0  ->  2250
2250
15 :
0  ->  2400
2400
16 :
0  ->  2550
2550
17 :
0  ->  2700
2700
18 :
0  ->  2850
2850
19 :
Last
0  ->  3000
3000


In [24]:
len(image_sets_lsa)

20

## Training guided by LSA

In [25]:
len(label_sets_lsa)

20

In [26]:
print(model_dir)

C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10


In [27]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [28]:
models_lsa[0].evaluate(x_test,y_test)

94/94 [==============================] - 11s 13ms/step - loss: 0.4941 - accuracy: 0.8294


[0.5391883254051208, 0.8036666512489319]

In [29]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n=n+1

Epoch 1/20
2/2 [==============================] - 6s 820ms/step - loss: 2.3114 - accuracy: 0.2067 - val_loss: 0.9083 - val_accuracy: 0.6797
Epoch 2/20
2/2 [==============================] - 1s 810ms/step - loss: 1.4280 - accuracy: 0.4133 - val_loss: 0.8683 - val_accuracy: 0.7008
Epoch 3/20
2/2 [==============================] - 1s 806ms/step - loss: 1.5181 - accuracy: 0.4467 - val_loss: 0.8544 - val_accuracy: 0.7286
Epoch 4/20
2/2 [==============================] - 1s 813ms/step - loss: 1.3201 - accuracy: 0.4600 - val_loss: 0.9386 - val_accuracy: 0.6509
Epoch 5/20
2/2 [==============================] - 1s 851ms/step - loss: 1.2452 - accuracy: 0.5000 - val_loss: 0.8571 - val_accuracy: 0.6940
Epoch 6/20
2/2 [==============================] - 1s 831ms/step - loss: 1.1000 - accuracy: 0.5733 - val_loss: 0.8588 - val_accuracy: 0.6883
Epoch 7/20
2/2 [==============================] - 1s 844ms/step - loss: 1.6257 - accuracy: 0.4200 - val_loss: 1.0413 - val_accuracy: 0.6334
Epoch 8/20
2/2 [====

In [30]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

Epoch 1/20
3/3 [==============================] - 3s 729ms/step - loss: 2.3497 - accuracy: 0.2813 - val_loss: 1.1498 - val_accuracy: 0.6794
Epoch 2/20
3/3 [==============================] - 1s 529ms/step - loss: 1.4842 - accuracy: 0.3926 - val_loss: 0.9530 - val_accuracy: 0.6922
Epoch 3/20
3/3 [==============================] - 1s 518ms/step - loss: 1.3056 - accuracy: 0.4907 - val_loss: 0.9824 - val_accuracy: 0.6266
Epoch 4/20
3/3 [==============================] - 1s 522ms/step - loss: 1.2166 - accuracy: 0.5064 - val_loss: 1.0100 - val_accuracy: 0.6223
Epoch 5/20
3/3 [==============================] - 1s 520ms/step - loss: 1.3249 - accuracy: 0.4721 - val_loss: 0.8025 - val_accuracy: 0.7254
Epoch 6/20
3/3 [==============================] - 1s 616ms/step - loss: 1.0899 - accuracy: 0.5571 - val_loss: 0.8800 - val_accuracy: 0.6641
Epoch 7/20
3/3 [==============================] - 1s 630ms/step - loss: 1.1414 - accuracy: 0.5602 - val_loss: 0.9670 - val_accuracy: 0.6737
Epoch 8/20
3/3 [====

In [31]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
4/4 [==============================] - 4s 639ms/step - loss: 2.4638 - accuracy: 0.2245 - val_loss: 1.1279 - val_accuracy: 0.6223
Epoch 2/20
4/4 [==============================] - 2s 571ms/step - loss: 1.5057 - accuracy: 0.4075 - val_loss: 0.9439 - val_accuracy: 0.7211
Epoch 3/20
4/4 [==============================] - 2s 617ms/step - loss: 1.3229 - accuracy: 0.5013 - val_loss: 0.8765 - val_accuracy: 0.7143
Epoch 4/20
4/4 [==============================] - 2s 605ms/step - loss: 1.2813 - accuracy: 0.4762 - val_loss: 0.9577 - val_accuracy: 0.6687
Epoch 5/20
4/4 [==============================] - 2s 593ms/step - loss: 1.2601 - accuracy: 0.5016 - val_loss: 0.8699 - val_accuracy: 0.7154
Epoch 6/20
4/4 [==============================] - 2s 640ms/step - loss: 1.1364 - accuracy: 0.5505 - val_loss: 0.8555 - val_accuracy: 0.7086
Epoch 7/20
4/4 [==============================] - 2s 664ms/step - loss: 1.0797 - accuracy: 0.5448 - val_loss: 0.8538 - val_accuracy: 0.7093
Epoch 8/20
4/4 [==

In [32]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
5/5 [==============================] - 5s 694ms/step - loss: 2.3064 - accuracy: 0.2659 - val_loss: 1.1607 - val_accuracy: 0.6049
Epoch 2/20
5/5 [==============================] - 2s 559ms/step - loss: 1.4150 - accuracy: 0.4124 - val_loss: 1.0609 - val_accuracy: 0.5603
Epoch 3/20
5/5 [==============================] - 3s 580ms/step - loss: 1.4938 - accuracy: 0.4202 - val_loss: 0.9782 - val_accuracy: 0.6626
Epoch 4/20
5/5 [==============================] - 3s 587ms/step - loss: 1.2924 - accuracy: 0.4842 - val_loss: 1.0682 - val_accuracy: 0.6387
Epoch 5/20
5/5 [==============================] - 2s 555ms/step - loss: 1.2918 - accuracy: 0.4984 - val_loss: 0.9176 - val_accuracy: 0.7179
Epoch 6/20
5/5 [==============================] - 2s 561ms/step - loss: 1.1961 - accuracy: 0.4996 - val_loss: 0.8454 - val_accuracy: 0.7011
Epoch 7/20
5/5 [==============================] - 2s 562ms/step - loss: 1.1730 - accuracy: 0.5168 - val_loss: 0.8515 - val_accuracy: 0.6972
Epoch 8/20
5/5 [==

In [33]:
print(n) #4
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
6/6 [==============================] - 5s 640ms/step - loss: 2.2424 - accuracy: 0.3151 - val_loss: 1.0200 - val_accuracy: 0.7507
Epoch 2/20
6/6 [==============================] - 3s 498ms/step - loss: 1.3411 - accuracy: 0.4400 - val_loss: 0.8737 - val_accuracy: 0.7254
Epoch 3/20
6/6 [==============================] - 3s 512ms/step - loss: 1.2489 - accuracy: 0.4943 - val_loss: 0.9707 - val_accuracy: 0.6769
Epoch 4/20
6/6 [==============================] - 3s 513ms/step - loss: 1.2419 - accuracy: 0.4828 - val_loss: 0.9243 - val_accuracy: 0.6758
Epoch 5/20
6/6 [==============================] - 3s 507ms/step - loss: 1.1090 - accuracy: 0.5658 - val_loss: 0.7864 - val_accuracy: 0.7336
Epoch 6/20
6/6 [==============================] - 3s 499ms/step - loss: 1.0260 - accuracy: 0.5960 - val_loss: 1.0295 - val_accuracy: 0.6398
Epoch 7/20
6/6 [==============================] - 3s 548ms/step - loss: 1.1364 - accuracy: 0.5281 - val_loss: 1.0356 - val_accuracy: 0.6280
Epoch 8/20
6/6 [==

In [34]:
n=5
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
8/8 [==============================] - 5s 423ms/step - loss: 1.7050 - accuracy: 0.4129 - val_loss: 1.3767 - val_accuracy: 0.5410
Epoch 2/20
8/8 [==============================] - 3s 377ms/step - loss: 1.3541 - accuracy: 0.4482 - val_loss: 1.5317 - val_accuracy: 0.4579
Epoch 3/20
8/8 [==============================] - 3s 391ms/step - loss: 1.1510 - accuracy: 0.5597 - val_loss: 1.1400 - val_accuracy: 0.5556
Epoch 4/20
8/8 [==============================] - 3s 429ms/step - loss: 1.1164 - accuracy: 0.5621 - val_loss: 1.0543 - val_accuracy: 0.6434
Epoch 5/20
8/8 [==============================] - 3s 406ms/step - loss: 1.1060 - accuracy: 0.5929 - val_loss: 1.4416 - val_accuracy: 0.5364
Epoch 6/20
8/8 [==============================] - 3s 359ms/step - loss: 1.5588 - accuracy: 0.5274 - val_loss: 0.8546 - val_accuracy: 0.7247
Epoch 7/20
8/8 [==============================] - 3s 374ms/step - loss: 0.9881 - accuracy: 0.6150 - val_loss: 0.8237 - val_accuracy: 0.6969
Epoch 8/20
8/8 [==

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
9/9 [==============================] - 5s 380ms/step - loss: 1.7357 - accuracy: 0.3504 - val_loss: 0.9869 - val_accuracy: 0.6394
Epoch 2/20
9/9 [==============================] - 3s 365ms/step - loss: 1.2891 - accuracy: 0.4891 - val_loss: 0.8394 - val_accuracy: 0.7183
Epoch 3/20
9/9 [==============================] - 3s 353ms/step - loss: 1.1348 - accuracy: 0.5479 - val_loss: 0.9909 - val_accuracy: 0.6241
Epoch 4/20
9/9 [==============================] - 3s 361ms/step - loss: 1.2815 - accuracy: 0.4847 - val_loss: 0.8903 - val_accuracy: 0.6583
Epoch 5/20
9/9 [==============================] - 3s 358ms/step - loss: 1.0552 - accuracy: 0.5628 - val_loss: 0.8671 - val_accuracy: 0.6890
Epoch 6/20
9/9 [==============================] - 3s 358ms/step - loss: 0.9697 - accuracy: 0.6158 - val_loss: 0.8249 - val_accuracy: 0.6726
Epoch 7/20
9/9 [==============================] - 3s 355ms/step - loss: 0.9624 - accuracy: 0.5932 - val_loss: 0.8831 - val_accuracy: 0.6630
Epoch 8/20
9/9 [==

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
10/10 [==============================] - 5s 370ms/step - loss: 1.8291 - accuracy: 0.3560 - val_loss: 1.0526 - val_accuracy: 0.6444
Epoch 2/20
10/10 [==============================] - 3s 338ms/step - loss: 1.2089 - accuracy: 0.4771 - val_loss: 1.0011 - val_accuracy: 0.6024
Epoch 3/20
10/10 [==============================] - 3s 350ms/step - loss: 1.0917 - accuracy: 0.5469 - val_loss: 0.8749 - val_accuracy: 0.6769
Epoch 4/20
10/10 [==============================] - 3s 341ms/step - loss: 1.0979 - accuracy: 0.5412 - val_loss: 1.0923 - val_accuracy: 0.6066
Epoch 5/20
10/10 [==============================] - 3s 341ms/step - loss: 1.0958 - accuracy: 0.5776 - val_loss: 0.9122 - val_accuracy: 0.6890
Epoch 6/20
10/10 [==============================] - 3s 344ms/step - loss: 1.0055 - accuracy: 0.5783 - val_loss: 0.8137 - val_accuracy: 0.6826
Epoch 7/20
10/10 [==============================] - 3s 341ms/step - loss: 1.0107 - accuracy: 0.5778 - val_loss: 0.8775 - val_accuracy: 0.6619
Epoc

In [37]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
11/11 [==============================] - 5s 358ms/step - loss: 1.5760 - accuracy: 0.4116 - val_loss: 0.9068 - val_accuracy: 0.7364
Epoch 2/20
11/11 [==============================] - 4s 337ms/step - loss: 1.1244 - accuracy: 0.5557 - val_loss: 0.8933 - val_accuracy: 0.7193
Epoch 3/20
11/11 [==============================] - 4s 332ms/step - loss: 1.0681 - accuracy: 0.5468 - val_loss: 0.7925 - val_accuracy: 0.7104
Epoch 4/20
11/11 [==============================] - 4s 333ms/step - loss: 1.0197 - accuracy: 0.5967 - val_loss: 0.8649 - val_accuracy: 0.6723
Epoch 5/20
11/11 [==============================] - 4s 331ms/step - loss: 1.0292 - accuracy: 0.5796 - val_loss: 0.9188 - val_accuracy: 0.6883
Epoch 6/20
11/11 [==============================] - 4s 331ms/step - loss: 1.0263 - accuracy: 0.5871 - val_loss: 0.7892 - val_accuracy: 0.7115
Epoch 7/20
11/11 [==============================] - 4s 330ms/step - loss: 0.9637 - accuracy: 0.6231 - val_loss: 0.7985 - val_accuracy: 0.7036
Epoc

In [38]:
print(n)#9
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
12/12 [==============================] - 5s 340ms/step - loss: 1.6189 - accuracy: 0.4201 - val_loss: 0.8278 - val_accuracy: 0.7457
Epoch 2/20
12/12 [==============================] - 4s 325ms/step - loss: 1.0734 - accuracy: 0.5564 - val_loss: 0.7851 - val_accuracy: 0.7218
Epoch 3/20
12/12 [==============================] - 4s 322ms/step - loss: 1.0716 - accuracy: 0.5653 - val_loss: 0.7529 - val_accuracy: 0.7268
Epoch 4/20
12/12 [==============================] - 4s 324ms/step - loss: 0.9667 - accuracy: 0.6027 - val_loss: 0.8125 - val_accuracy: 0.7286
Epoch 5/20
12/12 [==============================] - 4s 321ms/step - loss: 0.9502 - accuracy: 0.6150 - val_loss: 0.7866 - val_accuracy: 0.7240
Epoch 6/20
12/12 [==============================] - 4s 320ms/step - loss: 0.9751 - accuracy: 0.6082 - val_loss: 0.7412 - val_accuracy: 0.7432
Epoch 7/20
12/12 [==============================] - 4s 323ms/step - loss: 0.8520 - accuracy: 0.6602 - val_loss: 0.7376 - val_accuracy: 0.7372
Epoc

In [39]:
## print(n)
n=10
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

Epoch 1/20
13/13 [==============================] - 6s 331ms/step - loss: 1.6137 - accuracy: 0.3887 - val_loss: 0.8419 - val_accuracy: 0.7361
Epoch 2/20
13/13 [==============================] - 4s 316ms/step - loss: 1.1141 - accuracy: 0.5304 - val_loss: 0.8120 - val_accuracy: 0.7193
Epoch 3/20
13/13 [==============================] - 4s 316ms/step - loss: 1.0533 - accuracy: 0.5601 - val_loss: 0.8467 - val_accuracy: 0.7108
Epoch 4/20
13/13 [==============================] - 4s 317ms/step - loss: 1.0073 - accuracy: 0.5758 - val_loss: 0.8345 - val_accuracy: 0.6986
Epoch 5/20
13/13 [==============================] - 4s 313ms/step - loss: 1.0310 - accuracy: 0.5992 - val_loss: 0.8155 - val_accuracy: 0.7340
Epoch 6/20
13/13 [==============================] - 4s 314ms/step - loss: 0.9566 - accuracy: 0.6274 - val_loss: 0.7499 - val_accuracy: 0.7389
Epoch 7/20
13/13 [==============================] - 4s 320ms/step - loss: 0.8928 - accuracy: 0.6442 - val_loss: 0.8147 - val_accuracy: 0.7322
Epoch 

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
15/15 [==============================] - 6s 306ms/step - loss: 1.4058 - accuracy: 0.4667 - val_loss: 1.0679 - val_accuracy: 0.6245
Epoch 2/20
15/15 [==============================] - 4s 285ms/step - loss: 1.1188 - accuracy: 0.5591 - val_loss: 0.9682 - val_accuracy: 0.6583
Epoch 3/20
15/15 [==============================] - 4s 295ms/step - loss: 1.0654 - accuracy: 0.5621 - val_loss: 0.8828 - val_accuracy: 0.7033
Epoch 4/20
15/15 [==============================] - 4s 289ms/step - loss: 0.9979 - accuracy: 0.6009 - val_loss: 1.0068 - val_accuracy: 0.6120
Epoch 5/20
15/15 [==============================] - 4s 288ms/step - loss: 1.1106 - accuracy: 0.5677 - val_loss: 0.8334 - val_accuracy: 0.6947
Epoch 6/20
15/15 [==============================] - 4s 289ms/step - loss: 1.0743 - accuracy: 0.5810 - val_loss: 0.8309 - val_accuracy: 0.7147
Epoch 7/20
15/15 [==============================] - 4s 288ms/step - loss: 0.9617 - accuracy: 0.6283 - val_loss: 1.1748 - val_accuracy: 0.5667
Epo

In [41]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
16/16 [==============================] - 6s 302ms/step - loss: 1.3930 - accuracy: 0.4869 - val_loss: 0.8600 - val_accuracy: 0.7397
Epoch 2/20
16/16 [==============================] - 4s 287ms/step - loss: 1.0952 - accuracy: 0.5556 - val_loss: 0.7602 - val_accuracy: 0.7393
Epoch 3/20
16/16 [==============================] - 5s 292ms/step - loss: 1.0452 - accuracy: 0.5867 - val_loss: 0.8898 - val_accuracy: 0.6623
Epoch 4/20
16/16 [==============================] - 4s 282ms/step - loss: 0.9751 - accuracy: 0.5988 - val_loss: 0.8922 - val_accuracy: 0.6990
Epoch 5/20
16/16 [==============================] - 4s 286ms/step - loss: 0.9694 - accuracy: 0.6116 - val_loss: 1.0293 - val_accuracy: 0.6305
Epoch 6/20
16/16 [==============================] - 5s 287ms/step - loss: 1.1038 - accuracy: 0.5756 - val_loss: 0.7574 - val_accuracy: 0.7397
Epoch 7/20
16/16 [==============================] - 4s 285ms/step - loss: 0.8479 - accuracy: 0.6512 - val_loss: 0.7193 - val_accuracy: 0.7461
Epo

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
17/17 [==============================] - 6s 295ms/step - loss: 1.5545 - accuracy: 0.4589 - val_loss: 0.7992 - val_accuracy: 0.7443
Epoch 2/20
17/17 [==============================] - 5s 286ms/step - loss: 1.0710 - accuracy: 0.5556 - val_loss: 0.7101 - val_accuracy: 0.7471
Epoch 3/20
17/17 [==============================] - 5s 284ms/step - loss: 1.0087 - accuracy: 0.5806 - val_loss: 0.6891 - val_accuracy: 0.7561
Epoch 4/20
17/17 [==============================] - 5s 285ms/step - loss: 0.9640 - accuracy: 0.6063 - val_loss: 0.8770 - val_accuracy: 0.7001
Epoch 5/20
17/17 [==============================] - 5s 283ms/step - loss: 0.9403 - accuracy: 0.6269 - val_loss: 0.8350 - val_accuracy: 0.7193
Epoch 6/20
17/17 [==============================] - 5s 281ms/step - loss: 0.8930 - accuracy: 0.6390 - val_loss: 0.8831 - val_accuracy: 0.6776
Epoch 7/20
17/17 [==============================] - 5s 283ms/step - loss: 0.8622 - accuracy: 0.6542 - val_loss: 0.7239 - val_accuracy: 0.7354
Epo

In [43]:
print(n)#14
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
18/18 [==============================] - 7s 292ms/step - loss: 1.3670 - accuracy: 0.4812 - val_loss: 0.8326 - val_accuracy: 0.6961
Epoch 2/20
18/18 [==============================] - 5s 280ms/step - loss: 1.1096 - accuracy: 0.5657 - val_loss: 0.7636 - val_accuracy: 0.7447
Epoch 3/20
18/18 [==============================] - 5s 280ms/step - loss: 0.9436 - accuracy: 0.6273 - val_loss: 0.8215 - val_accuracy: 0.7140
Epoch 4/20
18/18 [==============================] - 5s 286ms/step - loss: 0.9534 - accuracy: 0.6135 - val_loss: 0.6799 - val_accuracy: 0.7664
Epoch 5/20
18/18 [==============================] - 5s 281ms/step - loss: 0.8865 - accuracy: 0.6446 - val_loss: 0.6537 - val_accuracy: 0.7728
Epoch 6/20
18/18 [==============================] - 5s 285ms/step - loss: 0.8517 - accuracy: 0.6527 - val_loss: 0.6949 - val_accuracy: 0.7643
Epoch 7/20
18/18 [==============================] - 5s 278ms/step - loss: 0.7875 - accuracy: 0.6825 - val_loss: 0.6774 - val_accuracy: 0.7607
Epo

In [44]:
n=15
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
19/19 [==============================] - 7s 291ms/step - loss: 1.3288 - accuracy: 0.5121 - val_loss: 0.7735 - val_accuracy: 0.7447
Epoch 2/20
19/19 [==============================] - 5s 280ms/step - loss: 0.9819 - accuracy: 0.6027 - val_loss: 0.7552 - val_accuracy: 0.7111
Epoch 3/20
19/19 [==============================] - 5s 278ms/step - loss: 0.9319 - accuracy: 0.6247 - val_loss: 0.8479 - val_accuracy: 0.7022
Epoch 4/20
19/19 [==============================] - 5s 279ms/step - loss: 0.9004 - accuracy: 0.6353 - val_loss: 0.6980 - val_accuracy: 0.7489
Epoch 5/20
19/19 [==============================] - 5s 279ms/step - loss: 0.8911 - accuracy: 0.6402 - val_loss: 0.7173 - val_accuracy: 0.7489
Epoch 6/20
19/19 [==============================] - 5s 278ms/step - loss: 0.8617 - accuracy: 0.6651 - val_loss: 0.6396 - val_accuracy: 0.7750
Epoch 7/20
19/19 [==============================] - 5s 292ms/step - loss: 0.8082 - accuracy: 0.6839 - val_loss: 0.6862 - val_accuracy: 0.7650
Epo

In [45]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
20/20 [==============================] - 7s 288ms/step - loss: 1.3222 - accuracy: 0.5017 - val_loss: 0.6983 - val_accuracy: 0.7611
Epoch 2/20
20/20 [==============================] - 6s 284ms/step - loss: 0.9771 - accuracy: 0.6060 - val_loss: 0.6347 - val_accuracy: 0.7885
Epoch 3/20
20/20 [==============================] - 6s 289ms/step - loss: 0.9079 - accuracy: 0.6296 - val_loss: 0.6403 - val_accuracy: 0.7764
Epoch 4/20
20/20 [==============================] - 6s 288ms/step - loss: 0.9346 - accuracy: 0.6483 - val_loss: 0.7879 - val_accuracy: 0.7147
Epoch 5/20
20/20 [==============================] - 6s 288ms/step - loss: 0.8619 - accuracy: 0.6598 - val_loss: 0.7748 - val_accuracy: 0.7504
Epoch 6/20
20/20 [==============================] - 6s 287ms/step - loss: 0.8414 - accuracy: 0.6723 - val_loss: 0.8041 - val_accuracy: 0.7343
Epoch 7/20
20/20 [==============================] - 6s 287ms/step - loss: 0.8191 - accuracy: 0.6721 - val_loss: 0.6238 - val_accuracy: 0.7828
Epo

In [46]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
22/22 [==============================] - 8s 287ms/step - loss: 1.4994 - accuracy: 0.4785 - val_loss: 1.0360 - val_accuracy: 0.6548
Epoch 2/20
22/22 [==============================] - 6s 276ms/step - loss: 1.0490 - accuracy: 0.5713 - val_loss: 0.7579 - val_accuracy: 0.7536
Epoch 3/20
22/22 [==============================] - 6s 275ms/step - loss: 0.9346 - accuracy: 0.6451 - val_loss: 0.7799 - val_accuracy: 0.7611
Epoch 4/20
22/22 [==============================] - 6s 274ms/step - loss: 0.9135 - accuracy: 0.6565 - val_loss: 0.7739 - val_accuracy: 0.7397
Epoch 5/20
22/22 [==============================] - 6s 273ms/step - loss: 0.9083 - accuracy: 0.6275 - val_loss: 0.8643 - val_accuracy: 0.6715
Epoch 6/20
22/22 [==============================] - 6s 272ms/step - loss: 0.9192 - accuracy: 0.6366 - val_loss: 0.7015 - val_accuracy: 0.7571
Epoch 7/20
22/22 [==============================] - 6s 274ms/step - loss: 0.8256 - accuracy: 0.6715 - val_loss: 0.7000 - val_accuracy: 0.7628
Epo

In [47]:
time.sleep(400)

In [48]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
23/23 [==============================] - 5s 172ms/step - loss: 1.2185 - accuracy: 0.5337 - val_loss: 0.6973 - val_accuracy: 0.7689
Epoch 2/20
23/23 [==============================] - 4s 157ms/step - loss: 0.9157 - accuracy: 0.6530 - val_loss: 0.6852 - val_accuracy: 0.7596
Epoch 3/20
23/23 [==============================] - 4s 160ms/step - loss: 0.8866 - accuracy: 0.6483 - val_loss: 0.8366 - val_accuracy: 0.6986
Epoch 4/20
23/23 [==============================] - 4s 165ms/step - loss: 0.9116 - accuracy: 0.6394 - val_loss: 1.2210 - val_accuracy: 0.6123
Epoch 5/20
23/23 [==============================] - 4s 167ms/step - loss: 0.9694 - accuracy: 0.6284 - val_loss: 0.7809 - val_accuracy: 0.7186
Epoch 6/20
23/23 [==============================] - 4s 171ms/step - loss: 0.8466 - accuracy: 0.6800 - val_loss: 0.6223 - val_accuracy: 0.7785
Epoch 7/20
23/23 [==============================] - 4s 171ms/step - loss: 0.7944 - accuracy: 0.7005 - val_loss: 0.6685 - val_accuracy: 0.7743
Epo

In [ ]:
time.sleep(400)

In [49]:
print(n)#19
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
24/24 [==============================] - 8s 291ms/step - loss: 1.2437 - accuracy: 0.5513 - val_loss: 0.6416 - val_accuracy: 0.7707
Epoch 2/20
24/24 [==============================] - 7s 276ms/step - loss: 0.9281 - accuracy: 0.6377 - val_loss: 0.6501 - val_accuracy: 0.7821
Epoch 3/20
24/24 [==============================] - 6s 274ms/step - loss: 0.8638 - accuracy: 0.6583 - val_loss: 0.7757 - val_accuracy: 0.7261
Epoch 4/20
24/24 [==============================] - 7s 274ms/step - loss: 0.8519 - accuracy: 0.6598 - val_loss: 0.6149 - val_accuracy: 0.7853
Epoch 5/20
24/24 [==============================] - 6s 271ms/step - loss: 0.8169 - accuracy: 0.6890 - val_loss: 0.8396 - val_accuracy: 0.7051
Epoch 6/20
24/24 [==============================] - 6s 271ms/step - loss: 0.8105 - accuracy: 0.6824 - val_loss: 0.5784 - val_accuracy: 0.7974
Epoch 7/20
24/24 [==============================] - 6s 272ms/step - loss: 0.7788 - accuracy: 0.7075 - val_loss: 0.6760 - val_accuracy: 0.7625
Epo

In [50]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
new_model_lsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_lsa_e2"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_lsa_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_lsa_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_lsa_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_lsa_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_lsa_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_lsa_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_lsa_e2_6\assets
Model has been saved
INFO:t

In [51]:
import gc

In [52]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [53]:
gc.collect()

474400

In [ ]:
loading = False

models_lsa = []

if loading:
    for i in range(10,20):
        #model_lsa_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_lsa_e1_"+str(i)
        print(model_lsa_dir)
        model =utils.My_model(dataset,True,model_lsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_lsa.append(model)
       
    

## Training guided by DSA

In [ ]:
dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"

In [54]:
dsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/dsa_values.npy"

In [55]:
dsa_values = np.load(dsa_direction)[len(x_train):] 
print(len(dsa_values))

3000


In [56]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_adversary_training)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_adversary_training)

In [57]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  150
150
1 :
0  ->  300
300
2 :
0  ->  450
450
3 :
0  ->  600
600
4 :
0  ->  750
750
5 :
0  ->  900
900
6 :
0  ->  1050
1050
7 :
0  ->  1200
1200
8 :
0  ->  1350
1350
9 :
0  ->  1500
1500
10 :
0  ->  1650
1650
11 :
0  ->  1800
1800
12 :
0  ->  1950
1950
13 :
0  ->  2100
2100
14 :
0  ->  2250
2250
15 :
0  ->  2400
2400
16 :
0  ->  2550
2550
17 :
0  ->  2700
2700
18 :
0  ->  2850
2850
19 :
Last
0  ->  3000
3000


In [58]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [59]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
2/2 [==============================] - 3s 2s/step - loss: 2.3117 - accuracy: 0.2151 - val_loss: 1.2505 - val_accuracy: 0.5688
Epoch 2/20
2/2 [==============================] - 2s 1s/step - loss: 1.6251 - accuracy: 0.3353 - val_loss: 0.9261 - val_accuracy: 0.6926
Epoch 3/20
2/2 [==============================] - 2s 1s/step - loss: 1.4064 - accuracy: 0.4533 - val_loss: 1.1297 - val_accuracy: 0.6084
Epoch 4/20
2/2 [==============================] - 2s 1s/step - loss: 1.4472 - accuracy: 0.4595 - val_loss: 0.8094 - val_accuracy: 0.7086
Epoch 5/20
2/2 [==============================] - 2s 2s/step - loss: 1.1622 - accuracy: 0.5768 - val_loss: 0.7996 - val_accuracy: 0.7076
Epoch 6/20
2/2 [==============================] - 2s 1s/step - loss: 1.0917 - accuracy: 0.6005 - val_loss: 0.8245 - val_accuracy: 0.6983
Epoch 7/20
2/2 [==============================] - 2s 2s/step - loss: 1.0659 - accuracy: 0.6394 - val_loss: 0.7728 - val_accuracy: 0.7161
Epoch 8/20
2/2 [=======================

In [60]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
3/3 [==============================] - 4s 1s/step - loss: 2.3482 - accuracy: 0.2714 - val_loss: 0.9585 - val_accuracy: 0.7158
Epoch 2/20
3/3 [==============================] - 2s 864ms/step - loss: 1.4024 - accuracy: 0.3700 - val_loss: 0.9528 - val_accuracy: 0.6658
Epoch 3/20
3/3 [==============================] - 2s 871ms/step - loss: 1.3721 - accuracy: 0.4159 - val_loss: 0.8326 - val_accuracy: 0.7257
Epoch 4/20
3/3 [==============================] - 2s 872ms/step - loss: 1.2104 - accuracy: 0.5185 - val_loss: 0.9614 - val_accuracy: 0.5984
Epoch 5/20
3/3 [==============================] - 2s 869ms/step - loss: 1.2721 - accuracy: 0.5140 - val_loss: 0.7601 - val_accuracy: 0.7443
Epoch 6/20
3/3 [==============================] - 2s 879ms/step - loss: 1.1772 - accuracy: 0.5144 - val_loss: 0.7693 - val_accuracy: 0.7307
Epoch 7/20
3/3 [==============================] - 2s 865ms/step - loss: 1.1168 - accuracy: 0.5785 - val_loss: 0.8096 - val_accuracy: 0.7193
Epoch 8/20
3/3 [=====

In [61]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
4/4 [==============================] - 4s 769ms/step - loss: 1.9807 - accuracy: 0.2588 - val_loss: 1.1779 - val_accuracy: 0.5835
Epoch 2/20
4/4 [==============================] - 2s 676ms/step - loss: 1.4232 - accuracy: 0.3917 - val_loss: 0.8443 - val_accuracy: 0.6826
Epoch 3/20
4/4 [==============================] - 2s 667ms/step - loss: 1.2086 - accuracy: 0.5085 - val_loss: 0.8667 - val_accuracy: 0.6623
Epoch 4/20
4/4 [==============================] - 2s 669ms/step - loss: 1.1461 - accuracy: 0.5446 - val_loss: 1.0107 - val_accuracy: 0.6091
Epoch 5/20
4/4 [==============================] - 2s 667ms/step - loss: 1.2103 - accuracy: 0.4978 - val_loss: 0.8692 - val_accuracy: 0.6626
Epoch 6/20
4/4 [==============================] - 2s 672ms/step - loss: 1.0336 - accuracy: 0.5814 - val_loss: 0.8665 - val_accuracy: 0.6965
Epoch 7/20
4/4 [==============================] - 2s 664ms/step - loss: 1.1304 - accuracy: 0.5644 - val_loss: 0.8586 - val_accuracy: 0.6665
Epoch 8/20
4/4 [==

In [62]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
5/5 [==============================] - 4s 652ms/step - loss: 1.9782 - accuracy: 0.2664 - val_loss: 1.0429 - val_accuracy: 0.6612
Epoch 2/20
5/5 [==============================] - 3s 578ms/step - loss: 1.3647 - accuracy: 0.4377 - val_loss: 0.8289 - val_accuracy: 0.7076
Epoch 3/20
5/5 [==============================] - 3s 571ms/step - loss: 1.2541 - accuracy: 0.4568 - val_loss: 0.8010 - val_accuracy: 0.7257
Epoch 4/20
5/5 [==============================] - 2s 566ms/step - loss: 1.2460 - accuracy: 0.4870 - val_loss: 0.8204 - val_accuracy: 0.7022
Epoch 5/20
5/5 [==============================] - 3s 569ms/step - loss: 1.1432 - accuracy: 0.5214 - val_loss: 0.8425 - val_accuracy: 0.7058
Epoch 6/20
5/5 [==============================] - 2s 563ms/step - loss: 1.0427 - accuracy: 0.5687 - val_loss: 0.8781 - val_accuracy: 0.6837
Epoch 7/20
5/5 [==============================] - 3s 569ms/step - loss: 1.1300 - accuracy: 0.5476 - val_loss: 0.8108 - val_accuracy: 0.7193
Epoch 8/20
5/5 [==

In [63]:
print(n)#4

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
6/6 [==============================] - 5s 571ms/step - loss: 1.8974 - accuracy: 0.3079 - val_loss: 1.0536 - val_accuracy: 0.7090
Epoch 2/20
6/6 [==============================] - 3s 520ms/step - loss: 1.4006 - accuracy: 0.4372 - val_loss: 0.9083 - val_accuracy: 0.7261
Epoch 3/20
6/6 [==============================] - 3s 503ms/step - loss: 1.2335 - accuracy: 0.4854 - val_loss: 0.8938 - val_accuracy: 0.7047
Epoch 4/20
6/6 [==============================] - 3s 505ms/step - loss: 1.2456 - accuracy: 0.5139 - val_loss: 0.8936 - val_accuracy: 0.6776
Epoch 5/20
6/6 [==============================] - 3s 513ms/step - loss: 1.1181 - accuracy: 0.5218 - val_loss: 0.8849 - val_accuracy: 0.6908
Epoch 6/20
6/6 [==============================] - 3s 510ms/step - loss: 1.1383 - accuracy: 0.5344 - val_loss: 0.7731 - val_accuracy: 0.7240
Epoch 7/20
6/6 [==============================] - 3s 517ms/step - loss: 1.0621 - accuracy: 0.5714 - val_loss: 0.8856 - val_accuracy: 0.6972
Epoch 8/20
6/6 [==

In [64]:
n=5
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
8/8 [==============================] - 5s 453ms/step - loss: 2.0015 - accuracy: 0.2449 - val_loss: 1.0643 - val_accuracy: 0.6330
Epoch 2/20
8/8 [==============================] - 3s 407ms/step - loss: 1.3947 - accuracy: 0.4186 - val_loss: 0.9510 - val_accuracy: 0.6598
Epoch 3/20
8/8 [==============================] - 3s 406ms/step - loss: 1.3434 - accuracy: 0.4224 - val_loss: 1.0871 - val_accuracy: 0.5749
Epoch 4/20
8/8 [==============================] - 3s 408ms/step - loss: 1.3578 - accuracy: 0.4463 - val_loss: 0.9418 - val_accuracy: 0.6890
Epoch 5/20
8/8 [==============================] - 3s 403ms/step - loss: 1.2583 - accuracy: 0.4869 - val_loss: 0.8874 - val_accuracy: 0.6391
Epoch 6/20
8/8 [==============================] - 3s 407ms/step - loss: 1.1392 - accuracy: 0.5287 - val_loss: 1.4407 - val_accuracy: 0.4747
Epoch 7/20
8/8 [==============================] - 3s 402ms/step - loss: 1.3031 - accuracy: 0.4690 - val_loss: 0.8906 - val_accuracy: 0.6733
Epoch 8/20
8/8 [==

In [65]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
9/9 [==============================] - 5s 433ms/step - loss: 2.1745 - accuracy: 0.2593 - val_loss: 1.1435 - val_accuracy: 0.6876
Epoch 2/20
9/9 [==============================] - 3s 388ms/step - loss: 1.4262 - accuracy: 0.3973 - val_loss: 1.0634 - val_accuracy: 0.6444
Epoch 3/20
9/9 [==============================] - 3s 391ms/step - loss: 1.3318 - accuracy: 0.4433 - val_loss: 0.9161 - val_accuracy: 0.6844
Epoch 4/20
9/9 [==============================] - 3s 393ms/step - loss: 1.2745 - accuracy: 0.4562 - val_loss: 0.8199 - val_accuracy: 0.7076
Epoch 5/20
9/9 [==============================] - 3s 386ms/step - loss: 1.2408 - accuracy: 0.4623 - val_loss: 1.2008 - val_accuracy: 0.5842
Epoch 6/20
9/9 [==============================] - 3s 399ms/step - loss: 1.2102 - accuracy: 0.5303 - val_loss: 0.8596 - val_accuracy: 0.6737
Epoch 7/20
9/9 [==============================] - 3s 386ms/step - loss: 1.1814 - accuracy: 0.5016 - val_loss: 0.8266 - val_accuracy: 0.7104
Epoch 8/20
9/9 [==

In [66]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
10/10 [==============================] - 5s 417ms/step - loss: 1.7565 - accuracy: 0.3386 - val_loss: 0.9363 - val_accuracy: 0.7222
Epoch 2/20
10/10 [==============================] - 4s 378ms/step - loss: 1.3113 - accuracy: 0.4310 - val_loss: 0.8175 - val_accuracy: 0.7175
Epoch 3/20
10/10 [==============================] - 4s 380ms/step - loss: 1.2477 - accuracy: 0.4799 - val_loss: 0.8435 - val_accuracy: 0.7200
Epoch 4/20
10/10 [==============================] - 4s 376ms/step - loss: 1.2016 - accuracy: 0.5083 - val_loss: 0.9629 - val_accuracy: 0.6473
Epoch 5/20
10/10 [==============================] - 4s 378ms/step - loss: 1.1848 - accuracy: 0.5070 - val_loss: 0.8813 - val_accuracy: 0.6755
Epoch 6/20
10/10 [==============================] - 4s 376ms/step - loss: 1.0659 - accuracy: 0.5269 - val_loss: 0.8317 - val_accuracy: 0.7140
Epoch 7/20
10/10 [==============================] - 4s 379ms/step - loss: 1.1099 - accuracy: 0.5293 - val_loss: 0.8469 - val_accuracy: 0.6594
Epoc

In [67]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
11/11 [==============================] - 6s 409ms/step - loss: 1.8189 - accuracy: 0.3258 - val_loss: 0.9027 - val_accuracy: 0.7065
Epoch 2/20
11/11 [==============================] - 4s 364ms/step - loss: 1.3049 - accuracy: 0.4467 - val_loss: 0.9257 - val_accuracy: 0.7022
Epoch 3/20
11/11 [==============================] - 4s 370ms/step - loss: 1.2124 - accuracy: 0.4957 - val_loss: 0.8856 - val_accuracy: 0.7015
Epoch 4/20
11/11 [==============================] - 4s 368ms/step - loss: 1.1523 - accuracy: 0.5368 - val_loss: 0.7634 - val_accuracy: 0.7500
Epoch 5/20
11/11 [==============================] - 4s 368ms/step - loss: 1.1031 - accuracy: 0.5206 - val_loss: 0.8278 - val_accuracy: 0.7111
Epoch 6/20
11/11 [==============================] - 4s 367ms/step - loss: 1.0563 - accuracy: 0.5567 - val_loss: 0.7374 - val_accuracy: 0.7504
Epoch 7/20
11/11 [==============================] - 4s 372ms/step - loss: 1.0409 - accuracy: 0.5769 - val_loss: 0.7899 - val_accuracy: 0.7279
Epoc

In [68]:
print(n)#9

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
12/12 [==============================] - 6s 391ms/step - loss: 2.1020 - accuracy: 0.3033 - val_loss: 0.8313 - val_accuracy: 0.7578
Epoch 2/20
12/12 [==============================] - 4s 361ms/step - loss: 1.2972 - accuracy: 0.4614 - val_loss: 0.7624 - val_accuracy: 0.7671
Epoch 3/20
12/12 [==============================] - 4s 362ms/step - loss: 1.1794 - accuracy: 0.5022 - val_loss: 0.8500 - val_accuracy: 0.7233
Epoch 4/20
12/12 [==============================] - 4s 357ms/step - loss: 1.2213 - accuracy: 0.5094 - val_loss: 0.7328 - val_accuracy: 0.7553
Epoch 5/20
12/12 [==============================] - 4s 358ms/step - loss: 1.1065 - accuracy: 0.5596 - val_loss: 0.7451 - val_accuracy: 0.7372
Epoch 6/20
12/12 [==============================] - 4s 360ms/step - loss: 1.0734 - accuracy: 0.5691 - val_loss: 1.1735 - val_accuracy: 0.6213
Epoch 7/20
12/12 [==============================] - 4s 362ms/step - loss: 1.1737 - accuracy: 0.5312 - val_loss: 0.7686 - val_accuracy: 0.7322
Epoc

In [69]:
n=10
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
13/13 [==============================] - 6s 382ms/step - loss: 1.7452 - accuracy: 0.3515 - val_loss: 0.8764 - val_accuracy: 0.7557
Epoch 2/20
13/13 [==============================] - 5s 359ms/step - loss: 1.2114 - accuracy: 0.4972 - val_loss: 0.9983 - val_accuracy: 0.6715
Epoch 3/20
13/13 [==============================] - 4s 354ms/step - loss: 1.1636 - accuracy: 0.5307 - val_loss: 0.8386 - val_accuracy: 0.7282
Epoch 4/20
13/13 [==============================] - 5s 356ms/step - loss: 1.0951 - accuracy: 0.5341 - val_loss: 0.9253 - val_accuracy: 0.6969
Epoch 5/20
13/13 [==============================] - 5s 358ms/step - loss: 1.0744 - accuracy: 0.5681 - val_loss: 0.7399 - val_accuracy: 0.7496
Epoch 6/20
13/13 [==============================] - 4s 351ms/step - loss: 1.0617 - accuracy: 0.5673 - val_loss: 0.7842 - val_accuracy: 0.7400
Epoch 7/20
13/13 [==============================] - 5s 356ms/step - loss: 1.0155 - accuracy: 0.5897 - val_loss: 0.7076 - val_accuracy: 0.7550
Epo

In [70]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
15/15 [==============================] - 7s 354ms/step - loss: 1.6433 - accuracy: 0.4161 - val_loss: 0.9930 - val_accuracy: 0.6398
Epoch 2/20
15/15 [==============================] - 5s 330ms/step - loss: 1.1717 - accuracy: 0.5355 - val_loss: 1.0492 - val_accuracy: 0.6223
Epoch 3/20
15/15 [==============================] - 5s 330ms/step - loss: 1.1585 - accuracy: 0.5452 - val_loss: 0.7589 - val_accuracy: 0.7343
Epoch 4/20
15/15 [==============================] - 5s 329ms/step - loss: 1.0611 - accuracy: 0.5643 - val_loss: 0.8723 - val_accuracy: 0.6787
Epoch 5/20
15/15 [==============================] - 5s 328ms/step - loss: 1.0520 - accuracy: 0.5832 - val_loss: 1.1306 - val_accuracy: 0.5253
Epoch 6/20
15/15 [==============================] - 5s 326ms/step - loss: 1.0293 - accuracy: 0.5721 - val_loss: 0.8238 - val_accuracy: 0.6701
Epoch 7/20
15/15 [==============================] - 5s 328ms/step - loss: 1.0517 - accuracy: 0.5788 - val_loss: 0.9738 - val_accuracy: 0.6523
Epo

In [71]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
16/16 [==============================] - 7s 353ms/step - loss: 1.5723 - accuracy: 0.4523 - val_loss: 0.9327 - val_accuracy: 0.6858
Epoch 2/20
16/16 [==============================] - 5s 324ms/step - loss: 1.1541 - accuracy: 0.5041 - val_loss: 0.7882 - val_accuracy: 0.7436
Epoch 3/20
16/16 [==============================] - 5s 332ms/step - loss: 1.1098 - accuracy: 0.5579 - val_loss: 0.7123 - val_accuracy: 0.7678
Epoch 4/20
16/16 [==============================] - 5s 325ms/step - loss: 1.0088 - accuracy: 0.5993 - val_loss: 0.7179 - val_accuracy: 0.7425
Epoch 5/20
16/16 [==============================] - 5s 326ms/step - loss: 0.9235 - accuracy: 0.6371 - val_loss: 0.7678 - val_accuracy: 0.7443
Epoch 6/20
16/16 [==============================] - 5s 323ms/step - loss: 0.9981 - accuracy: 0.6062 - val_loss: 0.6775 - val_accuracy: 0.7685
Epoch 7/20
16/16 [==============================] - 5s 326ms/step - loss: 0.9019 - accuracy: 0.6311 - val_loss: 1.0193 - val_accuracy: 0.6437
Epo

In [72]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
17/17 [==============================] - 7s 345ms/step - loss: 1.7172 - accuracy: 0.3949 - val_loss: 0.7870 - val_accuracy: 0.7183
Epoch 2/20
17/17 [==============================] - 5s 323ms/step - loss: 1.1299 - accuracy: 0.5482 - val_loss: 0.7141 - val_accuracy: 0.7596
Epoch 3/20
17/17 [==============================] - 6s 330ms/step - loss: 1.0045 - accuracy: 0.6052 - val_loss: 0.8077 - val_accuracy: 0.7083
Epoch 4/20
17/17 [==============================] - 5s 327ms/step - loss: 1.0224 - accuracy: 0.5984 - val_loss: 0.6530 - val_accuracy: 0.7625
Epoch 5/20
17/17 [==============================] - 5s 323ms/step - loss: 0.9925 - accuracy: 0.6022 - val_loss: 0.9305 - val_accuracy: 0.7001
Epoch 6/20
17/17 [==============================] - 6s 329ms/step - loss: 1.0001 - accuracy: 0.6158 - val_loss: 0.7190 - val_accuracy: 0.7550
Epoch 7/20
17/17 [==============================] - 5s 324ms/step - loss: 0.9496 - accuracy: 0.6455 - val_loss: 0.7465 - val_accuracy: 0.7354
Epo

In [73]:
print(n)#14

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
18/18 [==============================] - 8s 343ms/step - loss: 1.3804 - accuracy: 0.4890 - val_loss: 0.7741 - val_accuracy: 0.7450
Epoch 2/20
18/18 [==============================] - 6s 321ms/step - loss: 1.0558 - accuracy: 0.5620 - val_loss: 0.8940 - val_accuracy: 0.6712
Epoch 3/20
18/18 [==============================] - 6s 321ms/step - loss: 1.0555 - accuracy: 0.5809 - val_loss: 0.7492 - val_accuracy: 0.7400
Epoch 4/20
18/18 [==============================] - 6s 325ms/step - loss: 0.9270 - accuracy: 0.6362 - val_loss: 0.6643 - val_accuracy: 0.7767
Epoch 5/20
18/18 [==============================] - 6s 323ms/step - loss: 0.9632 - accuracy: 0.6086 - val_loss: 0.7470 - val_accuracy: 0.7443
Epoch 6/20
18/18 [==============================] - 6s 322ms/step - loss: 0.9682 - accuracy: 0.6111 - val_loss: 0.7318 - val_accuracy: 0.7525
Epoch 7/20
18/18 [==============================] - 6s 325ms/step - loss: 0.8716 - accuracy: 0.6506 - val_loss: 0.7103 - val_accuracy: 0.7500
Epo

In [74]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
19/19 [==============================] - 8s 341ms/step - loss: 1.3428 - accuracy: 0.5002 - val_loss: 0.7661 - val_accuracy: 0.7568
Epoch 2/20
19/19 [==============================] - 6s 320ms/step - loss: 1.0430 - accuracy: 0.5830 - val_loss: 0.7612 - val_accuracy: 0.7582
Epoch 3/20
19/19 [==============================] - 6s 321ms/step - loss: 1.0001 - accuracy: 0.5908 - val_loss: 0.6951 - val_accuracy: 0.7443
Epoch 4/20
19/19 [==============================] - 6s 324ms/step - loss: 0.9500 - accuracy: 0.6361 - val_loss: 0.7176 - val_accuracy: 0.7529
Epoch 5/20
19/19 [==============================] - 6s 323ms/step - loss: 0.9475 - accuracy: 0.6219 - val_loss: 0.7978 - val_accuracy: 0.7122
Epoch 6/20
19/19 [==============================] - 6s 322ms/step - loss: 0.9182 - accuracy: 0.6281 - val_loss: 0.6608 - val_accuracy: 0.7853
Epoch 7/20
19/19 [==============================] - 6s 323ms/step - loss: 0.8679 - accuracy: 0.6652 - val_loss: 0.6440 - val_accuracy: 0.7725
Epo

In [75]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
20/20 [==============================] - 8s 338ms/step - loss: 1.4456 - accuracy: 0.4647 - val_loss: 0.6721 - val_accuracy: 0.7739
Epoch 2/20
20/20 [==============================] - 6s 322ms/step - loss: 1.0035 - accuracy: 0.6114 - val_loss: 0.7019 - val_accuracy: 0.7650
Epoch 3/20
20/20 [==============================] - 6s 322ms/step - loss: 1.0074 - accuracy: 0.6077 - val_loss: 0.6891 - val_accuracy: 0.7586
Epoch 4/20
20/20 [==============================] - 6s 325ms/step - loss: 0.9651 - accuracy: 0.6227 - val_loss: 0.6916 - val_accuracy: 0.7553
Epoch 5/20
20/20 [==============================] - 6s 322ms/step - loss: 0.9228 - accuracy: 0.6362 - val_loss: 0.7208 - val_accuracy: 0.7589
Epoch 6/20
20/20 [==============================] - 6s 324ms/step - loss: 0.9269 - accuracy: 0.6241 - val_loss: 0.8416 - val_accuracy: 0.7079
Epoch 7/20
20/20 [==============================] - 6s 321ms/step - loss: 0.8651 - accuracy: 0.6592 - val_loss: 0.8377 - val_accuracy: 0.6983
Epo

In [76]:
time.sleep(400)

In [77]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
22/22 [==============================] - 5s 195ms/step - loss: 1.3407 - accuracy: 0.5123 - val_loss: 1.2265 - val_accuracy: 0.5506
Epoch 2/20
22/22 [==============================] - 4s 179ms/step - loss: 1.1092 - accuracy: 0.5711 - val_loss: 0.7953 - val_accuracy: 0.7504
Epoch 3/20
22/22 [==============================] - 4s 182ms/step - loss: 0.9418 - accuracy: 0.6282 - val_loss: 0.7303 - val_accuracy: 0.7432
Epoch 4/20
22/22 [==============================] - 4s 186ms/step - loss: 0.9155 - accuracy: 0.6533 - val_loss: 0.6962 - val_accuracy: 0.7600
Epoch 5/20
22/22 [==============================] - 4s 190ms/step - loss: 0.8518 - accuracy: 0.6835 - val_loss: 1.0960 - val_accuracy: 0.6216
Epoch 6/20
22/22 [==============================] - 4s 191ms/step - loss: 1.0059 - accuracy: 0.6293 - val_loss: 0.7937 - val_accuracy: 0.7086
Epoch 7/20
22/22 [==============================] - 4s 194ms/step - loss: 0.8454 - accuracy: 0.6683 - val_loss: 0.9710 - val_accuracy: 0.6865
Epo

In [ ]:
time.sleep(400)

In [78]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
23/23 [==============================] - 9s 324ms/step - loss: 1.3041 - accuracy: 0.5342 - val_loss: 0.7218 - val_accuracy: 0.7500
Epoch 2/20
23/23 [==============================] - 7s 304ms/step - loss: 0.9781 - accuracy: 0.6261 - val_loss: 0.7132 - val_accuracy: 0.7589
Epoch 3/20
23/23 [==============================] - 7s 308ms/step - loss: 0.9238 - accuracy: 0.6288 - val_loss: 0.7605 - val_accuracy: 0.7332
Epoch 4/20
23/23 [==============================] - 7s 304ms/step - loss: 0.8734 - accuracy: 0.6486 - val_loss: 0.6339 - val_accuracy: 0.7703
Epoch 5/20
23/23 [==============================] - 7s 304ms/step - loss: 0.8494 - accuracy: 0.6726 - val_loss: 0.6408 - val_accuracy: 0.7718
Epoch 6/20
23/23 [==============================] - 7s 305ms/step - loss: 0.8445 - accuracy: 0.6758 - val_loss: 0.7151 - val_accuracy: 0.7593
Epoch 7/20
23/23 [==============================] - 7s 304ms/step - loss: 0.7991 - accuracy: 0.6896 - val_loss: 0.6938 - val_accuracy: 0.7632
Epo

In [ ]:
time.sleep(400)

In [79]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
24/24 [==============================] - 9s 320ms/step - loss: 1.2411 - accuracy: 0.5613 - val_loss: 0.6423 - val_accuracy: 0.7739
Epoch 2/20
24/24 [==============================] - 7s 305ms/step - loss: 0.8857 - accuracy: 0.6572 - val_loss: 0.7984 - val_accuracy: 0.7293
Epoch 3/20
24/24 [==============================] - 7s 305ms/step - loss: 0.9041 - accuracy: 0.6548 - val_loss: 0.6611 - val_accuracy: 0.7725
Epoch 4/20
24/24 [==============================] - 7s 306ms/step - loss: 0.8414 - accuracy: 0.6770 - val_loss: 0.6564 - val_accuracy: 0.7668
Epoch 5/20
24/24 [==============================] - 7s 306ms/step - loss: 0.7829 - accuracy: 0.6936 - val_loss: 0.6318 - val_accuracy: 0.7803
Epoch 6/20
24/24 [==============================] - 8s 317ms/step - loss: 0.7806 - accuracy: 0.6959 - val_loss: 0.7205 - val_accuracy: 0.7636
Epoch 7/20
24/24 [==============================] - 7s 305ms/step - loss: 0.7272 - accuracy: 0.7150 - val_loss: 0.6278 - val_accuracy: 0.7839
Epo

In [ ]:
time.sleep(400)

In [80]:
new_model_dsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_dsa_e2"


i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dsa_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dsa_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dsa_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dsa_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dsa_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dsa_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dsa_e2_6\assets
Model has been saved
INFO:t

In [ ]:
loading = False

models_dsa = []

if loading:
    for i in range(20):
        model_dsa_dir = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2_"+str(i)
        print(model_dsa_dir)
        model =utils.My_model('gtsrb',True,model_dsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dsa.append(model)
       
    

In [81]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [82]:
gc.collect()

474387

## Uncertainty

In [83]:
dg_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)[len(x_train):] 
print(len(deep_gini_values))


3000


In [84]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_adversary_training)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_adversary_training)

In [85]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  150
150
1 :
0  ->  300
300
2 :
0  ->  450
450
3 :
0  ->  600
600
4 :
0  ->  750
750
5 :
0  ->  900
900
6 :
0  ->  1050
1050
7 :
0  ->  1200
1200
8 :
0  ->  1350
1350
9 :
0  ->  1500
1500
10 :
0  ->  1650
1650
11 :
0  ->  1800
1800
12 :
0  ->  1950
1950
13 :
0  ->  2100
2100
14 :
0  ->  2250
2250
15 :
0  ->  2400
2400
16 :
0  ->  2550
2550
17 :
0  ->  2700
2700
18 :
0  ->  2850
2850
19 :
Last
0  ->  3000
3000


In [86]:
print(model_dir)
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dg.append(model)

C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [87]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
2/2 [==============================] - 3s 2s/step - loss: 1.9199 - accuracy: 0.3193 - val_loss: 1.3738 - val_accuracy: 0.4929
Epoch 2/20
2/2 [==============================] - 2s 2s/step - loss: 1.9838 - accuracy: 0.2708 - val_loss: 0.9559 - val_accuracy: 0.6951
Epoch 3/20
2/2 [==============================] - 2s 2s/step - loss: 1.4772 - accuracy: 0.4251 - val_loss: 0.8170 - val_accuracy: 0.7389
Epoch 4/20
2/2 [==============================] - 2s 2s/step - loss: 1.2799 - accuracy: 0.5319 - val_loss: 0.7387 - val_accuracy: 0.7429
Epoch 5/20
2/2 [==============================] - 2s 1s/step - loss: 1.1867 - accuracy: 0.5078 - val_loss: 0.9057 - val_accuracy: 0.7008
Epoch 6/20
2/2 [==============================] - 2s 2s/step - loss: 1.4229 - accuracy: 0.4566 - val_loss: 0.8341 - val_accuracy: 0.7233
Epoch 7/20
2/2 [==============================] - 2s 1s/step - loss: 1.1186 - accuracy: 0.6076 - val_loss: 0.7380 - val_accuracy: 0.7525
Epoch 8/20
2/2 [=======================

In [88]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
3/3 [==============================] - 4s 1s/step - loss: 1.9006 - accuracy: 0.3608 - val_loss: 1.1781 - val_accuracy: 0.5303
Epoch 2/20
3/3 [==============================] - 2s 921ms/step - loss: 1.6463 - accuracy: 0.3393 - val_loss: 1.1710 - val_accuracy: 0.5895
Epoch 3/20
3/3 [==============================] - 2s 915ms/step - loss: 1.5517 - accuracy: 0.3839 - val_loss: 0.8202 - val_accuracy: 0.7275
Epoch 4/20
3/3 [==============================] - 2s 871ms/step - loss: 1.2741 - accuracy: 0.4863 - val_loss: 0.7847 - val_accuracy: 0.7357
Epoch 5/20
3/3 [==============================] - 2s 893ms/step - loss: 1.2013 - accuracy: 0.5517 - val_loss: 0.8188 - val_accuracy: 0.7240
Epoch 6/20
3/3 [==============================] - 2s 894ms/step - loss: 1.4151 - accuracy: 0.4553 - val_loss: 0.8299 - val_accuracy: 0.7158
Epoch 7/20
3/3 [==============================] - 2s 906ms/step - loss: 1.0729 - accuracy: 0.5623 - val_loss: 0.7629 - val_accuracy: 0.7154
Epoch 8/20
3/3 [=====

In [89]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
4/4 [==============================] - 4s 794ms/step - loss: 1.8062 - accuracy: 0.2799 - val_loss: 0.8991 - val_accuracy: 0.7029
Epoch 2/20
4/4 [==============================] - 2s 706ms/step - loss: 1.3779 - accuracy: 0.4567 - val_loss: 1.3052 - val_accuracy: 0.5592
Epoch 3/20
4/4 [==============================] - 2s 691ms/step - loss: 1.7572 - accuracy: 0.3811 - val_loss: 0.9323 - val_accuracy: 0.7161
Epoch 4/20
4/4 [==============================] - 2s 687ms/step - loss: 1.2692 - accuracy: 0.5193 - val_loss: 0.7931 - val_accuracy: 0.7122
Epoch 5/20
4/4 [==============================] - 2s 701ms/step - loss: 1.2045 - accuracy: 0.5209 - val_loss: 0.7392 - val_accuracy: 0.7521
Epoch 6/20
4/4 [==============================] - 2s 679ms/step - loss: 1.1107 - accuracy: 0.5794 - val_loss: 0.7180 - val_accuracy: 0.7507
Epoch 7/20
4/4 [==============================] - 2s 689ms/step - loss: 1.0748 - accuracy: 0.5405 - val_loss: 0.7763 - val_accuracy: 0.7400
Epoch 8/20
4/4 [==

In [90]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
5/5 [==============================] - 4s 660ms/step - loss: 1.8310 - accuracy: 0.2981 - val_loss: 1.0123 - val_accuracy: 0.6302
Epoch 2/20
5/5 [==============================] - 3s 590ms/step - loss: 1.4954 - accuracy: 0.3940 - val_loss: 0.8202 - val_accuracy: 0.7443
Epoch 3/20
5/5 [==============================] - 3s 588ms/step - loss: 1.3105 - accuracy: 0.4666 - val_loss: 0.6976 - val_accuracy: 0.7607
Epoch 4/20
5/5 [==============================] - 3s 588ms/step - loss: 1.2349 - accuracy: 0.5202 - val_loss: 0.7887 - val_accuracy: 0.7090
Epoch 5/20
5/5 [==============================] - 3s 581ms/step - loss: 1.2826 - accuracy: 0.4971 - val_loss: 0.7698 - val_accuracy: 0.7343
Epoch 6/20
5/5 [==============================] - 3s 593ms/step - loss: 1.0862 - accuracy: 0.5803 - val_loss: 0.7901 - val_accuracy: 0.7097
Epoch 7/20
5/5 [==============================] - 3s 588ms/step - loss: 1.1043 - accuracy: 0.5528 - val_loss: 0.7484 - val_accuracy: 0.7489
Epoch 8/20
5/5 [==

In [91]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
6/6 [==============================] - 5s 593ms/step - loss: 1.6444 - accuracy: 0.3528 - val_loss: 1.0073 - val_accuracy: 0.6412
Epoch 2/20
6/6 [==============================] - 3s 535ms/step - loss: 1.3936 - accuracy: 0.4292 - val_loss: 0.7998 - val_accuracy: 0.7118
Epoch 3/20
6/6 [==============================] - 3s 532ms/step - loss: 1.3463 - accuracy: 0.4366 - val_loss: 0.8091 - val_accuracy: 0.7293
Epoch 4/20
6/6 [==============================] - 3s 519ms/step - loss: 1.2074 - accuracy: 0.5064 - val_loss: 0.7981 - val_accuracy: 0.7300
Epoch 5/20
6/6 [==============================] - 3s 529ms/step - loss: 1.1862 - accuracy: 0.5138 - val_loss: 0.7165 - val_accuracy: 0.7529
Epoch 6/20
6/6 [==============================] - 3s 526ms/step - loss: 1.1098 - accuracy: 0.5480 - val_loss: 0.7088 - val_accuracy: 0.7496
Epoch 7/20
6/6 [==============================] - 3s 525ms/step - loss: 1.0788 - accuracy: 0.5353 - val_loss: 0.6647 - val_accuracy: 0.7689
Epoch 8/20
6/6 [==

In [92]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
8/8 [==============================] - 5s 464ms/step - loss: 1.7491 - accuracy: 0.3408 - val_loss: 0.8684 - val_accuracy: 0.7101
Epoch 2/20
8/8 [==============================] - 3s 416ms/step - loss: 1.2880 - accuracy: 0.4631 - val_loss: 1.5039 - val_accuracy: 0.4387
Epoch 3/20
8/8 [==============================] - 3s 420ms/step - loss: 1.4991 - accuracy: 0.4089 - val_loss: 0.7793 - val_accuracy: 0.7254
Epoch 4/20
8/8 [==============================] - 3s 417ms/step - loss: 1.2661 - accuracy: 0.5125 - val_loss: 1.0892 - val_accuracy: 0.5838
Epoch 5/20
8/8 [==============================] - 3s 416ms/step - loss: 1.2502 - accuracy: 0.4978 - val_loss: 0.8372 - val_accuracy: 0.7022
Epoch 6/20
8/8 [==============================] - 3s 416ms/step - loss: 1.2097 - accuracy: 0.5246 - val_loss: 1.1557 - val_accuracy: 0.6016
Epoch 7/20
8/8 [==============================] - 3s 416ms/step - loss: 1.2530 - accuracy: 0.5285 - val_loss: 1.2783 - val_accuracy: 0.6362
Epoch 8/20
8/8 [==

In [93]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
9/9 [==============================] - 5s 443ms/step - loss: 1.4231 - accuracy: 0.4090 - val_loss: 0.9818 - val_accuracy: 0.6865
Epoch 2/20
9/9 [==============================] - 3s 402ms/step - loss: 1.2171 - accuracy: 0.4774 - val_loss: 1.0417 - val_accuracy: 0.6234
Epoch 3/20
9/9 [==============================] - 3s 396ms/step - loss: 1.3957 - accuracy: 0.4576 - val_loss: 0.7322 - val_accuracy: 0.7493
Epoch 4/20
9/9 [==============================] - 3s 401ms/step - loss: 1.0334 - accuracy: 0.5674 - val_loss: 0.7490 - val_accuracy: 0.7475
Epoch 5/20
9/9 [==============================] - 3s 403ms/step - loss: 1.2435 - accuracy: 0.5003 - val_loss: 0.6861 - val_accuracy: 0.7539
Epoch 6/20
9/9 [==============================] - 3s 405ms/step - loss: 1.0193 - accuracy: 0.5791 - val_loss: 0.7387 - val_accuracy: 0.7486
Epoch 7/20
9/9 [==============================] - 3s 399ms/step - loss: 1.0509 - accuracy: 0.5583 - val_loss: 0.6989 - val_accuracy: 0.7500
Epoch 8/20
9/9 [==

In [94]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
10/10 [==============================] - 6s 429ms/step - loss: 1.6890 - accuracy: 0.3691 - val_loss: 0.9091 - val_accuracy: 0.6887
Epoch 2/20
10/10 [==============================] - 4s 395ms/step - loss: 1.2668 - accuracy: 0.4798 - val_loss: 0.8116 - val_accuracy: 0.7247
Epoch 3/20
10/10 [==============================] - 4s 402ms/step - loss: 1.1423 - accuracy: 0.5010 - val_loss: 0.6871 - val_accuracy: 0.7571
Epoch 4/20
10/10 [==============================] - 4s 399ms/step - loss: 1.1616 - accuracy: 0.5249 - val_loss: 0.6683 - val_accuracy: 0.7689
Epoch 5/20
10/10 [==============================] - 4s 393ms/step - loss: 1.0805 - accuracy: 0.5725 - val_loss: 0.7063 - val_accuracy: 0.7447
Epoch 6/20
10/10 [==============================] - 4s 397ms/step - loss: 1.0204 - accuracy: 0.5917 - val_loss: 0.8271 - val_accuracy: 0.6937
Epoch 7/20
10/10 [==============================] - 4s 392ms/step - loss: 1.0783 - accuracy: 0.5878 - val_loss: 0.7634 - val_accuracy: 0.7236
Epoc

In [95]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
11/11 [==============================] - 6s 420ms/step - loss: 1.5311 - accuracy: 0.3992 - val_loss: 0.7696 - val_accuracy: 0.7611
Epoch 2/20
11/11 [==============================] - 4s 388ms/step - loss: 1.1886 - accuracy: 0.5094 - val_loss: 0.7039 - val_accuracy: 0.7493
Epoch 3/20
11/11 [==============================] - 4s 391ms/step - loss: 1.1793 - accuracy: 0.5136 - val_loss: 0.6733 - val_accuracy: 0.7853
Epoch 4/20
11/11 [==============================] - 4s 381ms/step - loss: 1.0678 - accuracy: 0.5652 - val_loss: 0.7586 - val_accuracy: 0.7286
Epoch 5/20
11/11 [==============================] - 4s 382ms/step - loss: 1.0685 - accuracy: 0.5455 - val_loss: 0.7128 - val_accuracy: 0.7529
Epoch 6/20
11/11 [==============================] - 4s 385ms/step - loss: 1.0822 - accuracy: 0.5583 - val_loss: 0.7362 - val_accuracy: 0.7404
Epoch 7/20
11/11 [==============================] - 4s 384ms/step - loss: 1.0143 - accuracy: 0.5962 - val_loss: 0.7154 - val_accuracy: 0.7429
Epoc

In [96]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
12/12 [==============================] - 6s 409ms/step - loss: 1.4508 - accuracy: 0.4387 - val_loss: 0.7612 - val_accuracy: 0.7625
Epoch 2/20
12/12 [==============================] - 4s 379ms/step - loss: 1.1690 - accuracy: 0.5256 - val_loss: 0.7395 - val_accuracy: 0.7407
Epoch 3/20
12/12 [==============================] - 4s 382ms/step - loss: 1.1415 - accuracy: 0.5195 - val_loss: 0.6798 - val_accuracy: 0.7671
Epoch 4/20
12/12 [==============================] - 4s 380ms/step - loss: 1.0912 - accuracy: 0.5634 - val_loss: 0.6817 - val_accuracy: 0.7489
Epoch 5/20
12/12 [==============================] - 4s 377ms/step - loss: 1.0415 - accuracy: 0.5694 - val_loss: 0.6821 - val_accuracy: 0.7671
Epoch 6/20
12/12 [==============================] - 4s 379ms/step - loss: 0.9683 - accuracy: 0.6088 - val_loss: 0.6992 - val_accuracy: 0.7489
Epoch 7/20
12/12 [==============================] - 4s 378ms/step - loss: 0.9553 - accuracy: 0.6025 - val_loss: 0.6549 - val_accuracy: 0.7643
Epoc

In [97]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
13/13 [==============================] - 7s 400ms/step - loss: 1.4185 - accuracy: 0.4285 - val_loss: 0.7549 - val_accuracy: 0.7628
Epoch 2/20
13/13 [==============================] - 5s 368ms/step - loss: 1.1303 - accuracy: 0.5325 - val_loss: 0.6781 - val_accuracy: 0.7718
Epoch 3/20
13/13 [==============================] - 5s 368ms/step - loss: 1.1083 - accuracy: 0.5613 - val_loss: 0.8107 - val_accuracy: 0.7065
Epoch 4/20
13/13 [==============================] - 5s 369ms/step - loss: 1.0044 - accuracy: 0.5962 - val_loss: 0.6748 - val_accuracy: 0.7753
Epoch 5/20
13/13 [==============================] - 5s 370ms/step - loss: 1.0080 - accuracy: 0.5983 - val_loss: 0.7419 - val_accuracy: 0.7325
Epoch 6/20
13/13 [==============================] - 5s 369ms/step - loss: 1.0504 - accuracy: 0.5733 - val_loss: 0.6906 - val_accuracy: 0.7500
Epoch 7/20
13/13 [==============================] - 5s 366ms/step - loss: 0.9095 - accuracy: 0.6473 - val_loss: 0.6730 - val_accuracy: 0.7785
Epo

In [98]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
15/15 [==============================] - 7s 366ms/step - loss: 1.4884 - accuracy: 0.4280 - val_loss: 0.7945 - val_accuracy: 0.7225
Epoch 2/20
15/15 [==============================] - 5s 336ms/step - loss: 1.1122 - accuracy: 0.5470 - val_loss: 0.7575 - val_accuracy: 0.7297
Epoch 3/20
15/15 [==============================] - 5s 336ms/step - loss: 1.1463 - accuracy: 0.5274 - val_loss: 0.8185 - val_accuracy: 0.6979
Epoch 4/20
15/15 [==============================] - 5s 339ms/step - loss: 1.0804 - accuracy: 0.5573 - val_loss: 0.8681 - val_accuracy: 0.6844
Epoch 5/20
15/15 [==============================] - 5s 345ms/step - loss: 1.0206 - accuracy: 0.5854 - val_loss: 0.6598 - val_accuracy: 0.7728
Epoch 6/20
15/15 [==============================] - 5s 338ms/step - loss: 0.9723 - accuracy: 0.6040 - val_loss: 0.7602 - val_accuracy: 0.7282
Epoch 7/20
15/15 [==============================] - 5s 336ms/step - loss: 0.9089 - accuracy: 0.6309 - val_loss: 0.6914 - val_accuracy: 0.7582
Epo

In [99]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
16/16 [==============================] - 7s 360ms/step - loss: 1.4361 - accuracy: 0.4610 - val_loss: 0.9329 - val_accuracy: 0.6765
Epoch 2/20
16/16 [==============================] - 5s 339ms/step - loss: 1.1507 - accuracy: 0.5476 - val_loss: 0.9101 - val_accuracy: 0.7072
Epoch 3/20
16/16 [==============================] - 5s 339ms/step - loss: 1.0816 - accuracy: 0.5664 - val_loss: 0.7484 - val_accuracy: 0.7332
Epoch 4/20
16/16 [==============================] - 5s 337ms/step - loss: 1.0482 - accuracy: 0.5854 - val_loss: 0.7260 - val_accuracy: 0.7336
Epoch 5/20
16/16 [==============================] - 5s 341ms/step - loss: 0.9392 - accuracy: 0.6178 - val_loss: 0.7467 - val_accuracy: 0.7418
Epoch 6/20
16/16 [==============================] - 5s 339ms/step - loss: 0.9811 - accuracy: 0.5973 - val_loss: 0.6738 - val_accuracy: 0.7685
Epoch 7/20
16/16 [==============================] - 5s 334ms/step - loss: 0.9236 - accuracy: 0.6148 - val_loss: 0.7130 - val_accuracy: 0.7439
Epo

In [100]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
17/17 [==============================] - 7s 358ms/step - loss: 1.4253 - accuracy: 0.4627 - val_loss: 0.7210 - val_accuracy: 0.7607
Epoch 2/20
17/17 [==============================] - 6s 334ms/step - loss: 1.0782 - accuracy: 0.5833 - val_loss: 0.6770 - val_accuracy: 0.7739
Epoch 3/20
17/17 [==============================] - 6s 343ms/step - loss: 0.9893 - accuracy: 0.6121 - val_loss: 0.6900 - val_accuracy: 0.7632
Epoch 4/20
17/17 [==============================] - 6s 333ms/step - loss: 1.0242 - accuracy: 0.5913 - val_loss: 0.7303 - val_accuracy: 0.7382
Epoch 5/20
17/17 [==============================] - 6s 335ms/step - loss: 1.0466 - accuracy: 0.5920 - val_loss: 0.7552 - val_accuracy: 0.7368
Epoch 6/20
17/17 [==============================] - 6s 335ms/step - loss: 0.9320 - accuracy: 0.6338 - val_loss: 0.7201 - val_accuracy: 0.7529
Epoch 7/20
17/17 [==============================] - 6s 331ms/step - loss: 0.9060 - accuracy: 0.6373 - val_loss: 0.5952 - val_accuracy: 0.7910
Epo

In [101]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
18/18 [==============================] - 8s 357ms/step - loss: 1.3926 - accuracy: 0.4817 - val_loss: 0.7425 - val_accuracy: 0.7386
Epoch 2/20
18/18 [==============================] - 6s 336ms/step - loss: 1.0790 - accuracy: 0.5583 - val_loss: 0.6693 - val_accuracy: 0.7842
Epoch 3/20
18/18 [==============================] - 6s 333ms/step - loss: 0.9869 - accuracy: 0.6048 - val_loss: 0.7226 - val_accuracy: 0.7668
Epoch 4/20
18/18 [==============================] - 6s 333ms/step - loss: 0.9652 - accuracy: 0.6198 - val_loss: 0.6650 - val_accuracy: 0.7714
Epoch 5/20
18/18 [==============================] - 6s 332ms/step - loss: 0.9286 - accuracy: 0.6250 - val_loss: 0.7760 - val_accuracy: 0.7340
Epoch 6/20
18/18 [==============================] - 6s 334ms/step - loss: 0.9354 - accuracy: 0.6260 - val_loss: 0.6716 - val_accuracy: 0.7664
Epoch 7/20
18/18 [==============================] - 6s 332ms/step - loss: 0.8401 - accuracy: 0.6661 - val_loss: 0.7656 - val_accuracy: 0.7200
Epo

In [102]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
19/19 [==============================] - 8s 352ms/step - loss: 1.3465 - accuracy: 0.4729 - val_loss: 0.7053 - val_accuracy: 0.7778
Epoch 2/20
19/19 [==============================] - 6s 331ms/step - loss: 0.9912 - accuracy: 0.5872 - val_loss: 0.6591 - val_accuracy: 0.7710
Epoch 3/20
19/19 [==============================] - 6s 334ms/step - loss: 1.0089 - accuracy: 0.6016 - val_loss: 0.8176 - val_accuracy: 0.7197
Epoch 4/20
19/19 [==============================] - 6s 331ms/step - loss: 0.9734 - accuracy: 0.6166 - val_loss: 0.6303 - val_accuracy: 0.7874
Epoch 5/20
19/19 [==============================] - 6s 329ms/step - loss: 0.9107 - accuracy: 0.6374 - val_loss: 0.6843 - val_accuracy: 0.7636
Epoch 6/20
19/19 [==============================] - 6s 330ms/step - loss: 0.8694 - accuracy: 0.6555 - val_loss: 0.6319 - val_accuracy: 0.7718
Epoch 7/20
19/19 [==============================] - 6s 331ms/step - loss: 0.8723 - accuracy: 0.6623 - val_loss: 0.6107 - val_accuracy: 0.7782
Epo

In [103]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
20/20 [==============================] - 8s 357ms/step - loss: 1.2156 - accuracy: 0.5372 - val_loss: 0.6269 - val_accuracy: 0.7864
Epoch 2/20
20/20 [==============================] - 7s 347ms/step - loss: 0.9814 - accuracy: 0.6218 - val_loss: 0.6932 - val_accuracy: 0.7607
Epoch 3/20
20/20 [==============================] - 7s 334ms/step - loss: 0.9528 - accuracy: 0.6224 - val_loss: 0.6288 - val_accuracy: 0.7821
Epoch 4/20
20/20 [==============================] - 7s 331ms/step - loss: 0.9115 - accuracy: 0.6260 - val_loss: 0.6811 - val_accuracy: 0.7646
Epoch 5/20
20/20 [==============================] - 7s 331ms/step - loss: 0.8973 - accuracy: 0.6511 - val_loss: 0.7354 - val_accuracy: 0.7350
Epoch 6/20
20/20 [==============================] - 7s 330ms/step - loss: 0.8612 - accuracy: 0.6679 - val_loss: 0.6197 - val_accuracy: 0.7874
Epoch 7/20
20/20 [==============================] - 7s 329ms/step - loss: 0.7964 - accuracy: 0.6906 - val_loss: 0.8273 - val_accuracy: 0.7268
Epo

In [40]:
time.sleep(400)

In [104]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
22/22 [==============================] - 9s 338ms/step - loss: 1.2612 - accuracy: 0.5251 - val_loss: 0.8244 - val_accuracy: 0.6919
Epoch 2/20
22/22 [==============================] - 7s 317ms/step - loss: 1.0341 - accuracy: 0.6039 - val_loss: 1.3989 - val_accuracy: 0.5111
Epoch 3/20
22/22 [==============================] - 7s 317ms/step - loss: 1.2145 - accuracy: 0.5791 - val_loss: 0.6509 - val_accuracy: 0.7750
Epoch 4/20
22/22 [==============================] - 7s 316ms/step - loss: 0.8576 - accuracy: 0.6584 - val_loss: 0.8303 - val_accuracy: 0.7040
Epoch 5/20
22/22 [==============================] - 7s 319ms/step - loss: 0.9303 - accuracy: 0.6345 - val_loss: 0.6376 - val_accuracy: 0.7867
Epoch 6/20
22/22 [==============================] - 7s 321ms/step - loss: 0.8416 - accuracy: 0.6694 - val_loss: 0.6932 - val_accuracy: 0.7475
Epoch 7/20
22/22 [==============================] - 7s 317ms/step - loss: 0.8455 - accuracy: 0.6749 - val_loss: 0.6535 - val_accuracy: 0.7728
Epo

In [ ]:
time.sleep(400)

In [105]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
23/23 [==============================] - 9s 335ms/step - loss: 1.2219 - accuracy: 0.5451 - val_loss: 0.8022 - val_accuracy: 0.7240
Epoch 2/20
23/23 [==============================] - 7s 321ms/step - loss: 0.9317 - accuracy: 0.6307 - val_loss: 0.8195 - val_accuracy: 0.7108
Epoch 3/20
23/23 [==============================] - 7s 326ms/step - loss: 0.9239 - accuracy: 0.6422 - val_loss: 0.6529 - val_accuracy: 0.7660
Epoch 4/20
23/23 [==============================] - 7s 321ms/step - loss: 0.8785 - accuracy: 0.6675 - val_loss: 0.6687 - val_accuracy: 0.7696
Epoch 5/20
23/23 [==============================] - 7s 321ms/step - loss: 0.8490 - accuracy: 0.6614 - val_loss: 0.6371 - val_accuracy: 0.7732
Epoch 6/20
23/23 [==============================] - 7s 323ms/step - loss: 0.7831 - accuracy: 0.7005 - val_loss: 0.7150 - val_accuracy: 0.7414
Epoch 7/20
23/23 [==============================] - 7s 318ms/step - loss: 0.8131 - accuracy: 0.6925 - val_loss: 0.6475 - val_accuracy: 0.7700
Epo

In [ ]:
time.sleep(400)

In [106]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
24/24 [==============================] - 10s 338ms/step - loss: 1.1954 - accuracy: 0.5493 - val_loss: 0.7648 - val_accuracy: 0.7443
Epoch 2/20
24/24 [==============================] - 8s 323ms/step - loss: 0.9243 - accuracy: 0.6417 - val_loss: 0.6424 - val_accuracy: 0.7636
Epoch 3/20
24/24 [==============================] - 8s 324ms/step - loss: 0.9308 - accuracy: 0.6459 - val_loss: 0.6042 - val_accuracy: 0.7846
Epoch 4/20
24/24 [==============================] - 8s 320ms/step - loss: 0.7984 - accuracy: 0.6946 - val_loss: 0.6862 - val_accuracy: 0.7550
Epoch 5/20
24/24 [==============================] - 8s 322ms/step - loss: 0.8309 - accuracy: 0.6739 - val_loss: 0.7069 - val_accuracy: 0.7693
Epoch 6/20
24/24 [==============================] - 8s 321ms/step - loss: 0.8057 - accuracy: 0.6822 - val_loss: 0.7785 - val_accuracy: 0.7261
Epoch 7/20
24/24 [==============================] - 8s 321ms/step - loss: 0.8028 - accuracy: 0.6926 - val_loss: 0.7436 - val_accuracy: 0.7489
Ep

In [107]:
new_model_dg_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_dg_e2"

i=0

for model in models_dg:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dg_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dg_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dg_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dg_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dg_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dg_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_dg_e2_6\assets
Model has been saved
INFO:tensorfl

In [108]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [109]:

gc.collect()

417163

### Softmax 

In [110]:
#softmax values
se_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)[len(x_train):] 
print(len(se_values))



3000


In [111]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_adversary_training)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_adversary_training)

In [112]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  150
150
1 :
0  ->  300
300
2 :
0  ->  450
450
3 :
0  ->  600
600
4 :
0  ->  750
750
5 :
0  ->  900
900
6 :
0  ->  1050
1050
7 :
0  ->  1200
1200
8 :
0  ->  1350
1350
9 :
0  ->  1500
1500
10 :
0  ->  1650
1650
11 :
0  ->  1800
1800
12 :
0  ->  1950
1950
13 :
0  ->  2100
2100
14 :
0  ->  2250
2250
15 :
0  ->  2400
2400
16 :
0  ->  2550
2550
17 :
0  ->  2700
2700
18 :
0  ->  2850
2850
19 :
Last
0  ->  3000
3000


In [113]:
print(model_dir)
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_se.append(model)

C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [114]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
2/2 [==============================] - 4s 3s/step - loss: 1.9795 - accuracy: 0.2734 - val_loss: 1.3317 - val_accuracy: 0.5086
Epoch 2/20
2/2 [==============================] - 3s 2s/step - loss: 1.8031 - accuracy: 0.2567 - val_loss: 1.3375 - val_accuracy: 0.5314
Epoch 3/20
2/2 [==============================] - 3s 2s/step - loss: 1.6431 - accuracy: 0.3008 - val_loss: 1.1307 - val_accuracy: 0.6651
Epoch 4/20
2/2 [==============================] - 3s 2s/step - loss: 1.5007 - accuracy: 0.4455 - val_loss: 0.8932 - val_accuracy: 0.7044
Epoch 5/20
2/2 [==============================] - 3s 2s/step - loss: 1.3609 - accuracy: 0.4700 - val_loss: 0.9395 - val_accuracy: 0.6648
Epoch 6/20
2/2 [==============================] - 3s 2s/step - loss: 1.4197 - accuracy: 0.4154 - val_loss: 0.7967 - val_accuracy: 0.7307
Epoch 7/20
2/2 [==============================] - 3s 2s/step - loss: 1.1383 - accuracy: 0.5608 - val_loss: 1.0809 - val_accuracy: 0.6270
Epoch 8/20
2/2 [=======================

In [115]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
3/3 [==============================] - 5s 2s/step - loss: 2.9139 - accuracy: 0.2440 - val_loss: 1.1820 - val_accuracy: 0.5888
Epoch 2/20
3/3 [==============================] - 3s 1s/step - loss: 1.7922 - accuracy: 0.2833 - val_loss: 1.0285 - val_accuracy: 0.6555
Epoch 3/20
3/3 [==============================] - 3s 1s/step - loss: 1.4362 - accuracy: 0.4423 - val_loss: 0.9125 - val_accuracy: 0.6698
Epoch 4/20
3/3 [==============================] - 3s 1s/step - loss: 1.4034 - accuracy: 0.4349 - val_loss: 0.8291 - val_accuracy: 0.7607
Epoch 5/20
3/3 [==============================] - 3s 1s/step - loss: 1.2782 - accuracy: 0.5056 - val_loss: 0.7215 - val_accuracy: 0.7504
Epoch 6/20
3/3 [==============================] - 3s 1s/step - loss: 1.1817 - accuracy: 0.5348 - val_loss: 0.7784 - val_accuracy: 0.7229
Epoch 7/20
3/3 [==============================] - 3s 1s/step - loss: 1.1537 - accuracy: 0.5750 - val_loss: 0.7418 - val_accuracy: 0.7489
Epoch 8/20
3/3 [=======================

In [116]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
4/4 [==============================] - 5s 1s/step - loss: 1.9363 - accuracy: 0.3046 - val_loss: 1.1078 - val_accuracy: 0.6373
Epoch 2/20
4/4 [==============================] - 3s 1s/step - loss: 1.5959 - accuracy: 0.3664 - val_loss: 1.1171 - val_accuracy: 0.6345
Epoch 3/20
4/4 [==============================] - 3s 998ms/step - loss: 1.4800 - accuracy: 0.3779 - val_loss: 0.9157 - val_accuracy: 0.7068
Epoch 4/20
4/4 [==============================] - 3s 989ms/step - loss: 1.2737 - accuracy: 0.5333 - val_loss: 0.7778 - val_accuracy: 0.7290
Epoch 5/20
4/4 [==============================] - 3s 998ms/step - loss: 1.2892 - accuracy: 0.4641 - val_loss: 0.7691 - val_accuracy: 0.7322
Epoch 6/20
4/4 [==============================] - 3s 994ms/step - loss: 1.1292 - accuracy: 0.5340 - val_loss: 0.7171 - val_accuracy: 0.7546
Epoch 7/20
4/4 [==============================] - 3s 997ms/step - loss: 1.0663 - accuracy: 0.5906 - val_loss: 1.2203 - val_accuracy: 0.5877
Epoch 8/20
4/4 [========

In [117]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
5/5 [==============================] - 6s 940ms/step - loss: 1.7685 - accuracy: 0.3196 - val_loss: 1.1134 - val_accuracy: 0.6184
Epoch 2/20
5/5 [==============================] - 4s 868ms/step - loss: 1.4416 - accuracy: 0.3963 - val_loss: 0.8087 - val_accuracy: 0.7461
Epoch 3/20
5/5 [==============================] - 4s 854ms/step - loss: 1.3651 - accuracy: 0.4764 - val_loss: 0.7086 - val_accuracy: 0.7514
Epoch 4/20
5/5 [==============================] - 4s 853ms/step - loss: 1.1822 - accuracy: 0.5405 - val_loss: 0.7159 - val_accuracy: 0.7322
Epoch 5/20
5/5 [==============================] - 4s 854ms/step - loss: 1.2081 - accuracy: 0.5093 - val_loss: 0.7222 - val_accuracy: 0.7261
Epoch 6/20
5/5 [==============================] - 4s 854ms/step - loss: 1.1477 - accuracy: 0.5256 - val_loss: 0.7017 - val_accuracy: 0.7482
Epoch 7/20
5/5 [==============================] - 4s 852ms/step - loss: 1.1036 - accuracy: 0.5802 - val_loss: 0.8265 - val_accuracy: 0.6983
Epoch 8/20
5/5 [==

In [118]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
6/6 [==============================] - 6s 844ms/step - loss: 1.7653 - accuracy: 0.3433 - val_loss: 1.0099 - val_accuracy: 0.6840
Epoch 2/20
6/6 [==============================] - 4s 798ms/step - loss: 1.3454 - accuracy: 0.4864 - val_loss: 0.8698 - val_accuracy: 0.7111
Epoch 3/20
6/6 [==============================] - 4s 796ms/step - loss: 1.3536 - accuracy: 0.4533 - val_loss: 0.7025 - val_accuracy: 0.7525
Epoch 4/20
6/6 [==============================] - 4s 772ms/step - loss: 1.2085 - accuracy: 0.5151 - val_loss: 0.8393 - val_accuracy: 0.6979
Epoch 5/20
6/6 [==============================] - 4s 774ms/step - loss: 1.1157 - accuracy: 0.5530 - val_loss: 0.6577 - val_accuracy: 0.7632
Epoch 6/20
6/6 [==============================] - 4s 774ms/step - loss: 1.1462 - accuracy: 0.5483 - val_loss: 0.7121 - val_accuracy: 0.7429
Epoch 7/20
6/6 [==============================] - 4s 771ms/step - loss: 1.0409 - accuracy: 0.6045 - val_loss: 0.8501 - val_accuracy: 0.6591
Epoch 8/20
6/6 [==

In [119]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
8/8 [==============================] - 6s 661ms/step - loss: 1.8386 - accuracy: 0.3296 - val_loss: 0.9253 - val_accuracy: 0.7307
Epoch 2/20
8/8 [==============================] - 5s 618ms/step - loss: 1.4087 - accuracy: 0.4111 - val_loss: 0.7469 - val_accuracy: 0.7614
Epoch 3/20
8/8 [==============================] - 5s 619ms/step - loss: 1.2488 - accuracy: 0.4988 - val_loss: 1.0516 - val_accuracy: 0.5243
Epoch 4/20
8/8 [==============================] - 5s 610ms/step - loss: 1.3143 - accuracy: 0.4854 - val_loss: 0.8462 - val_accuracy: 0.6879
Epoch 5/20
8/8 [==============================] - 5s 613ms/step - loss: 1.2478 - accuracy: 0.5103 - val_loss: 0.7803 - val_accuracy: 0.7168
Epoch 6/20
8/8 [==============================] - 5s 613ms/step - loss: 1.2203 - accuracy: 0.5076 - val_loss: 1.0077 - val_accuracy: 0.6095
Epoch 7/20
8/8 [==============================] - 5s 612ms/step - loss: 1.1248 - accuracy: 0.5507 - val_loss: 0.8011 - val_accuracy: 0.7033
Epoch 8/20
8/8 [==

In [120]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
9/9 [==============================] - 7s 624ms/step - loss: 1.8091 - accuracy: 0.3273 - val_loss: 0.8969 - val_accuracy: 0.7272
Epoch 2/20
9/9 [==============================] - 5s 581ms/step - loss: 1.3477 - accuracy: 0.4504 - val_loss: 0.7376 - val_accuracy: 0.7372
Epoch 3/20
9/9 [==============================] - 5s 581ms/step - loss: 1.2139 - accuracy: 0.5216 - val_loss: 0.7276 - val_accuracy: 0.7425
Epoch 4/20
9/9 [==============================] - 5s 580ms/step - loss: 1.1435 - accuracy: 0.5517 - val_loss: 0.7196 - val_accuracy: 0.7471
Epoch 5/20
9/9 [==============================] - 5s 578ms/step - loss: 1.1407 - accuracy: 0.5512 - val_loss: 0.8234 - val_accuracy: 0.7058
Epoch 6/20
9/9 [==============================] - 5s 578ms/step - loss: 1.1106 - accuracy: 0.5621 - val_loss: 0.7007 - val_accuracy: 0.7600
Epoch 7/20
9/9 [==============================] - 5s 581ms/step - loss: 1.0536 - accuracy: 0.5822 - val_loss: 0.8474 - val_accuracy: 0.7011
Epoch 8/20
9/9 [==

In [121]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
10/10 [==============================] - 7s 611ms/step - loss: 1.6303 - accuracy: 0.3776 - val_loss: 0.9754 - val_accuracy: 0.6320
Epoch 2/20
10/10 [==============================] - 5s 559ms/step - loss: 1.3141 - accuracy: 0.4589 - val_loss: 0.7320 - val_accuracy: 0.7382
Epoch 3/20
10/10 [==============================] - 5s 560ms/step - loss: 1.2135 - accuracy: 0.4954 - val_loss: 0.7448 - val_accuracy: 0.7350
Epoch 4/20
10/10 [==============================] - 5s 552ms/step - loss: 1.1633 - accuracy: 0.5314 - val_loss: 0.6889 - val_accuracy: 0.7735
Epoch 5/20
10/10 [==============================] - 5s 556ms/step - loss: 1.0402 - accuracy: 0.5802 - val_loss: 0.8273 - val_accuracy: 0.6904
Epoch 6/20
10/10 [==============================] - 5s 562ms/step - loss: 1.0487 - accuracy: 0.5711 - val_loss: 0.8026 - val_accuracy: 0.7072
Epoch 7/20
10/10 [==============================] - 5s 559ms/step - loss: 1.0147 - accuracy: 0.5884 - val_loss: 0.7062 - val_accuracy: 0.7454
Epoc

In [122]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
11/11 [==============================] - 8s 569ms/step - loss: 1.5174 - accuracy: 0.4171 - val_loss: 0.7166 - val_accuracy: 0.7521
Epoch 2/20
11/11 [==============================] - 6s 535ms/step - loss: 1.1479 - accuracy: 0.5261 - val_loss: 0.7009 - val_accuracy: 0.7767
Epoch 3/20
11/11 [==============================] - 6s 529ms/step - loss: 1.1038 - accuracy: 0.5325 - val_loss: 0.8652 - val_accuracy: 0.6972
Epoch 4/20
11/11 [==============================] - 6s 530ms/step - loss: 1.1371 - accuracy: 0.5280 - val_loss: 0.7645 - val_accuracy: 0.7304
Epoch 5/20
11/11 [==============================] - 6s 537ms/step - loss: 1.0649 - accuracy: 0.5634 - val_loss: 0.7397 - val_accuracy: 0.7393
Epoch 6/20
11/11 [==============================] - 6s 532ms/step - loss: 1.0005 - accuracy: 0.5836 - val_loss: 0.6897 - val_accuracy: 0.7561
Epoch 7/20
11/11 [==============================] - 6s 526ms/step - loss: 1.0194 - accuracy: 0.6045 - val_loss: 0.7128 - val_accuracy: 0.7532
Epoc

In [123]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
12/12 [==============================] - 8s 563ms/step - loss: 1.5438 - accuracy: 0.3906 - val_loss: 0.7544 - val_accuracy: 0.7553
Epoch 2/20
12/12 [==============================] - 6s 531ms/step - loss: 1.1549 - accuracy: 0.5429 - val_loss: 0.8570 - val_accuracy: 0.6794
Epoch 3/20
12/12 [==============================] - 6s 527ms/step - loss: 1.1232 - accuracy: 0.5490 - val_loss: 0.6487 - val_accuracy: 0.7800
Epoch 4/20
12/12 [==============================] - 6s 529ms/step - loss: 1.0162 - accuracy: 0.5760 - val_loss: 0.8342 - val_accuracy: 0.6847
Epoch 5/20
12/12 [==============================] - 6s 529ms/step - loss: 1.0738 - accuracy: 0.5657 - val_loss: 0.7968 - val_accuracy: 0.7200
Epoch 6/20
12/12 [==============================] - 6s 527ms/step - loss: 1.0630 - accuracy: 0.5873 - val_loss: 0.7178 - val_accuracy: 0.7486
Epoch 7/20
12/12 [==============================] - 6s 528ms/step - loss: 1.0109 - accuracy: 0.6226 - val_loss: 0.7122 - val_accuracy: 0.7304
Epoc

In [124]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
13/13 [==============================] - 8s 550ms/step - loss: 1.4039 - accuracy: 0.4540 - val_loss: 0.7270 - val_accuracy: 0.7621
Epoch 2/20
13/13 [==============================] - 7s 526ms/step - loss: 1.1735 - accuracy: 0.5225 - val_loss: 1.1456 - val_accuracy: 0.6098
Epoch 3/20
13/13 [==============================] - 7s 522ms/step - loss: 1.1796 - accuracy: 0.5349 - val_loss: 0.6767 - val_accuracy: 0.7582
Epoch 4/20
13/13 [==============================] - 7s 528ms/step - loss: 1.0161 - accuracy: 0.5783 - val_loss: 0.6805 - val_accuracy: 0.7636
Epoch 5/20
13/13 [==============================] - 7s 524ms/step - loss: 0.9696 - accuracy: 0.6124 - val_loss: 0.8448 - val_accuracy: 0.7147
Epoch 6/20
13/13 [==============================] - 7s 525ms/step - loss: 0.9872 - accuracy: 0.6055 - val_loss: 0.7226 - val_accuracy: 0.7450
Epoch 7/20
13/13 [==============================] - 7s 525ms/step - loss: 0.9363 - accuracy: 0.6305 - val_loss: 0.6228 - val_accuracy: 0.7707
Epo

In [125]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
15/15 [==============================] - 9s 515ms/step - loss: 1.5479 - accuracy: 0.4215 - val_loss: 0.7049 - val_accuracy: 0.7589
Epoch 2/20
15/15 [==============================] - 7s 485ms/step - loss: 1.1903 - accuracy: 0.4989 - val_loss: 0.9150 - val_accuracy: 0.6819
Epoch 3/20
15/15 [==============================] - 7s 489ms/step - loss: 1.1340 - accuracy: 0.5594 - val_loss: 0.7091 - val_accuracy: 0.7436
Epoch 4/20
15/15 [==============================] - 7s 484ms/step - loss: 1.0212 - accuracy: 0.5965 - val_loss: 0.7663 - val_accuracy: 0.7375
Epoch 5/20
15/15 [==============================] - 7s 483ms/step - loss: 1.1349 - accuracy: 0.5831 - val_loss: 0.8944 - val_accuracy: 0.6637
Epoch 6/20
15/15 [==============================] - 7s 495ms/step - loss: 1.0280 - accuracy: 0.5976 - val_loss: 0.6973 - val_accuracy: 0.7589
Epoch 7/20
15/15 [==============================] - 7s 485ms/step - loss: 0.9606 - accuracy: 0.6110 - val_loss: 0.7399 - val_accuracy: 0.7400
Epo

In [126]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
16/16 [==============================] - 10s 526ms/step - loss: 1.5441 - accuracy: 0.4424 - val_loss: 0.7109 - val_accuracy: 0.7685
Epoch 2/20
16/16 [==============================] - 8s 498ms/step - loss: 1.1213 - accuracy: 0.5468 - val_loss: 0.8640 - val_accuracy: 0.7001
Epoch 3/20
16/16 [==============================] - 8s 495ms/step - loss: 1.0625 - accuracy: 0.5774 - val_loss: 0.7636 - val_accuracy: 0.7240
Epoch 4/20
16/16 [==============================] - 8s 514ms/step - loss: 1.0095 - accuracy: 0.6084 - val_loss: 0.6516 - val_accuracy: 0.7725
Epoch 5/20
16/16 [==============================] - 8s 505ms/step - loss: 0.9955 - accuracy: 0.6131 - val_loss: 1.0088 - val_accuracy: 0.6255
Epoch 6/20
16/16 [==============================] - 8s 532ms/step - loss: 1.0276 - accuracy: 0.5995 - val_loss: 0.7412 - val_accuracy: 0.7325
Epoch 7/20
16/16 [==============================] - 8s 496ms/step - loss: 0.9011 - accuracy: 0.6351 - val_loss: 0.6503 - val_accuracy: 0.7782
Ep

In [127]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
17/17 [==============================] - 10s 500ms/step - loss: 1.3858 - accuracy: 0.4818 - val_loss: 0.8421 - val_accuracy: 0.6851
Epoch 2/20
17/17 [==============================] - 8s 472ms/step - loss: 1.1077 - accuracy: 0.5469 - val_loss: 0.7128 - val_accuracy: 0.7582
Epoch 3/20
17/17 [==============================] - 8s 478ms/step - loss: 1.0081 - accuracy: 0.6025 - val_loss: 0.7019 - val_accuracy: 0.7600
Epoch 4/20
17/17 [==============================] - 8s 472ms/step - loss: 1.0148 - accuracy: 0.6061 - val_loss: 0.7664 - val_accuracy: 0.7297
Epoch 5/20
17/17 [==============================] - 8s 476ms/step - loss: 0.9877 - accuracy: 0.6089 - val_loss: 0.6936 - val_accuracy: 0.7657
Epoch 6/20
17/17 [==============================] - 8s 476ms/step - loss: 0.9239 - accuracy: 0.6284 - val_loss: 0.6410 - val_accuracy: 0.7803
Epoch 7/20
17/17 [==============================] - 8s 473ms/step - loss: 0.8859 - accuracy: 0.6514 - val_loss: 0.6216 - val_accuracy: 0.7782
Ep

In [128]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
18/18 [==============================] - 10s 503ms/step - loss: 1.2992 - accuracy: 0.4969 - val_loss: 0.7004 - val_accuracy: 0.7504
Epoch 2/20
18/18 [==============================] - 9s 480ms/step - loss: 1.0389 - accuracy: 0.5835 - val_loss: 0.6719 - val_accuracy: 0.7800
Epoch 3/20
18/18 [==============================] - 9s 481ms/step - loss: 0.9655 - accuracy: 0.6085 - val_loss: 0.8777 - val_accuracy: 0.7040
Epoch 4/20
18/18 [==============================] - 9s 495ms/step - loss: 0.9931 - accuracy: 0.6127 - val_loss: 0.6410 - val_accuracy: 0.7767
Epoch 5/20
18/18 [==============================] - 9s 483ms/step - loss: 0.9349 - accuracy: 0.6319 - val_loss: 0.7031 - val_accuracy: 0.7507
Epoch 6/20
18/18 [==============================] - 8s 477ms/step - loss: 0.8950 - accuracy: 0.6507 - val_loss: 0.6748 - val_accuracy: 0.7511
Epoch 7/20
18/18 [==============================] - 8s 474ms/step - loss: 0.8681 - accuracy: 0.6565 - val_loss: 0.6123 - val_accuracy: 0.7857
Ep

In [129]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
19/19 [==============================] - 11s 508ms/step - loss: 1.3063 - accuracy: 0.5189 - val_loss: 0.7719 - val_accuracy: 0.7311
Epoch 2/20
19/19 [==============================] - 9s 471ms/step - loss: 1.0362 - accuracy: 0.5954 - val_loss: 0.6642 - val_accuracy: 0.7821
Epoch 3/20
19/19 [==============================] - 9s 469ms/step - loss: 0.9795 - accuracy: 0.6147 - val_loss: 0.7479 - val_accuracy: 0.7343
Epoch 4/20
19/19 [==============================] - 9s 472ms/step - loss: 0.9546 - accuracy: 0.6214 - val_loss: 0.6581 - val_accuracy: 0.7814
Epoch 5/20
19/19 [==============================] - 9s 468ms/step - loss: 0.9333 - accuracy: 0.6358 - val_loss: 0.6833 - val_accuracy: 0.7678
Epoch 6/20
19/19 [==============================] - 9s 469ms/step - loss: 0.8586 - accuracy: 0.6489 - val_loss: 0.7463 - val_accuracy: 0.7382
Epoch 7/20
19/19 [==============================] - 9s 470ms/step - loss: 0.8630 - accuracy: 0.6542 - val_loss: 0.6635 - val_accuracy: 0.7767
Ep

In [130]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
20/20 [==============================] - 11s 493ms/step - loss: 1.4410 - accuracy: 0.4827 - val_loss: 0.6570 - val_accuracy: 0.7757
Epoch 2/20
20/20 [==============================] - 9s 479ms/step - loss: 1.0041 - accuracy: 0.6074 - val_loss: 0.8406 - val_accuracy: 0.7015
Epoch 3/20
20/20 [==============================] - 9s 477ms/step - loss: 0.9954 - accuracy: 0.6095 - val_loss: 0.6662 - val_accuracy: 0.7732
Epoch 4/20
20/20 [==============================] - 9s 478ms/step - loss: 0.9385 - accuracy: 0.6352 - val_loss: 0.6922 - val_accuracy: 0.7564
Epoch 5/20
20/20 [==============================] - 10s 480ms/step - loss: 0.9192 - accuracy: 0.6317 - val_loss: 0.6634 - val_accuracy: 0.7750
Epoch 6/20
20/20 [==============================] - 9s 477ms/step - loss: 0.8840 - accuracy: 0.6533 - val_loss: 0.6255 - val_accuracy: 0.7718
Epoch 7/20
20/20 [==============================] - 9s 479ms/step - loss: 0.8607 - accuracy: 0.6599 - val_loss: 0.6136 - val_accuracy: 0.7853
E

In [68]:
time.sleep(400)

In [131]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
22/22 [==============================] - 12s 471ms/step - loss: 1.2590 - accuracy: 0.5480 - val_loss: 1.4249 - val_accuracy: 0.5285
Epoch 2/20
22/22 [==============================] - 10s 452ms/step - loss: 1.0797 - accuracy: 0.6055 - val_loss: 0.6526 - val_accuracy: 0.7618
Epoch 3/20
22/22 [==============================] - 10s 456ms/step - loss: 0.9554 - accuracy: 0.6218 - val_loss: 0.8783 - val_accuracy: 0.7122
Epoch 4/20
22/22 [==============================] - 10s 455ms/step - loss: 1.0167 - accuracy: 0.6133 - val_loss: 0.8186 - val_accuracy: 0.7168
Epoch 5/20
22/22 [==============================] - 10s 455ms/step - loss: 0.9242 - accuracy: 0.6395 - val_loss: 0.8200 - val_accuracy: 0.6937
Epoch 6/20
22/22 [==============================] - 10s 452ms/step - loss: 0.8610 - accuracy: 0.6496 - val_loss: 0.8117 - val_accuracy: 0.7122
Epoch 7/20
22/22 [==============================] - 10s 458ms/step - loss: 0.8341 - accuracy: 0.6746 - val_loss: 0.8290 - val_accuracy: 0.6

In [ ]:
time.sleep(400)

In [132]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
23/23 [==============================] - 12s 480ms/step - loss: 1.2885 - accuracy: 0.5388 - val_loss: 0.8150 - val_accuracy: 0.7439
Epoch 2/20
23/23 [==============================] - 10s 454ms/step - loss: 0.9628 - accuracy: 0.6300 - val_loss: 0.7630 - val_accuracy: 0.7436
Epoch 3/20
23/23 [==============================] - 10s 451ms/step - loss: 0.8973 - accuracy: 0.6480 - val_loss: 0.7394 - val_accuracy: 0.7571
Epoch 4/20
23/23 [==============================] - 10s 458ms/step - loss: 0.8827 - accuracy: 0.6632 - val_loss: 0.7078 - val_accuracy: 0.7315
Epoch 5/20
23/23 [==============================] - 10s 450ms/step - loss: 0.8286 - accuracy: 0.6735 - val_loss: 0.6900 - val_accuracy: 0.7657
Epoch 6/20
23/23 [==============================] - 10s 456ms/step - loss: 0.8153 - accuracy: 0.6879 - val_loss: 0.7028 - val_accuracy: 0.7457
Epoch 7/20
23/23 [==============================] - 10s 456ms/step - loss: 0.7797 - accuracy: 0.6971 - val_loss: 0.7066 - val_accuracy: 0.7

In [ ]:
time.sleep(400)

In [133]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
24/24 [==============================] - 13s 483ms/step - loss: 1.1810 - accuracy: 0.5586 - val_loss: 0.6754 - val_accuracy: 0.7718
Epoch 2/20
24/24 [==============================] - 11s 462ms/step - loss: 0.9322 - accuracy: 0.6439 - val_loss: 0.6761 - val_accuracy: 0.7650
Epoch 3/20
24/24 [==============================] - 11s 462ms/step - loss: 0.8384 - accuracy: 0.6710 - val_loss: 0.6598 - val_accuracy: 0.7714
Epoch 4/20
24/24 [==============================] - 11s 469ms/step - loss: 0.8298 - accuracy: 0.6703 - val_loss: 0.6765 - val_accuracy: 0.7721
Epoch 5/20
24/24 [==============================] - 11s 465ms/step - loss: 0.7743 - accuracy: 0.7028 - val_loss: 0.6542 - val_accuracy: 0.7714
Epoch 6/20
24/24 [==============================] - 11s 464ms/step - loss: 0.7569 - accuracy: 0.7048 - val_loss: 0.7772 - val_accuracy: 0.7004
Epoch 7/20
24/24 [==============================] - 11s 470ms/step - loss: 0.7930 - accuracy: 0.6814 - val_loss: 0.6611 - val_accuracy: 0.7

In [134]:
len(models_se)

20

In [135]:
new_model_se_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_se_e2"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_se_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_se_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_se_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_se_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_se_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_se_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_se_e2_6\assets
Model has been saved
INFO:tensorfl

In [136]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [137]:
gc.collect()

513963

## Training guided by Random values

In [138]:
save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)[len(x_train):] 
print(len(random_indexes))


3000


In [139]:
len(random_indexes)

3000

In [140]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_adversary_training)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_adversary_training)

In [141]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  150
150
1 :
0  ->  300
300
2 :
0  ->  450
450
3 :
0  ->  600
600
4 :
0  ->  750
750
5 :
0  ->  900
900
6 :
0  ->  1050
1050
7 :
0  ->  1200
1200
8 :
0  ->  1350
1350
9 :
0  ->  1500
1500
10 :
0  ->  1650
1650
11 :
0  ->  1800
1800
12 :
0  ->  1950
1950
13 :
0  ->  2100
2100
14 :
0  ->  2250
2250
15 :
0  ->  2400
2400
16 :
0  ->  2550
2550
17 :
0  ->  2700
2700
18 :
0  ->  2850
2850
19 :
Last
0  ->  3000
3000


In [142]:
len(image_sets_random)

20

In [143]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [144]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
2/2 [==============================] - 4s 2s/step - loss: 1.5381 - accuracy: 0.5890 - val_loss: 1.0552 - val_accuracy: 0.6227
Epoch 2/20
2/2 [==============================] - 2s 2s/step - loss: 1.3319 - accuracy: 0.5008 - val_loss: 0.9613 - val_accuracy: 0.7193
Epoch 3/20
2/2 [==============================] - 2s 2s/step - loss: 1.0820 - accuracy: 0.6402 - val_loss: 0.9420 - val_accuracy: 0.6665
Epoch 4/20
2/2 [==============================] - 2s 2s/step - loss: 1.2531 - accuracy: 0.5608 - val_loss: 0.8356 - val_accuracy: 0.7175
Epoch 5/20
2/2 [==============================] - 2s 2s/step - loss: 1.0231 - accuracy: 0.6306 - val_loss: 0.7309 - val_accuracy: 0.7557
Epoch 6/20
2/2 [==============================] - 2s 2s/step - loss: 0.8011 - accuracy: 0.6833 - val_loss: 0.6775 - val_accuracy: 0.7743
Epoch 7/20
2/2 [==============================] - 2s 2s/step - loss: 0.7457 - accuracy: 0.6958 - val_loss: 0.6556 - val_accuracy: 0.7678
Epoch 8/20
2/2 [=======================

In [145]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
3/3 [==============================] - 4s 1s/step - loss: 1.6898 - accuracy: 0.4729 - val_loss: 0.9775 - val_accuracy: 0.6384
Epoch 2/20
3/3 [==============================] - 3s 1s/step - loss: 1.0595 - accuracy: 0.5752 - val_loss: 0.7616 - val_accuracy: 0.7464
Epoch 3/20
3/3 [==============================] - 3s 1s/step - loss: 1.0065 - accuracy: 0.5391 - val_loss: 0.7888 - val_accuracy: 0.7329
Epoch 4/20
3/3 [==============================] - 3s 1s/step - loss: 0.9054 - accuracy: 0.6293 - val_loss: 0.7575 - val_accuracy: 0.7247
Epoch 5/20
3/3 [==============================] - 3s 1s/step - loss: 0.8547 - accuracy: 0.6553 - val_loss: 0.6847 - val_accuracy: 0.7550
Epoch 6/20
3/3 [==============================] - 3s 1s/step - loss: 0.8044 - accuracy: 0.6726 - val_loss: 0.8020 - val_accuracy: 0.7065
Epoch 7/20
3/3 [==============================] - 3s 1s/step - loss: 0.9132 - accuracy: 0.6671 - val_loss: 0.7390 - val_accuracy: 0.7393
Epoch 8/20
3/3 [=======================

In [146]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
4/4 [==============================] - 5s 981ms/step - loss: 1.4648 - accuracy: 0.4991 - val_loss: 0.8282 - val_accuracy: 0.7126
Epoch 2/20
4/4 [==============================] - 3s 863ms/step - loss: 1.0027 - accuracy: 0.5887 - val_loss: 0.7835 - val_accuracy: 0.7361
Epoch 3/20
4/4 [==============================] - 3s 859ms/step - loss: 0.9774 - accuracy: 0.6498 - val_loss: 0.6450 - val_accuracy: 0.7707
Epoch 4/20
4/4 [==============================] - 3s 869ms/step - loss: 0.8328 - accuracy: 0.6902 - val_loss: 0.6457 - val_accuracy: 0.7703
Epoch 5/20
4/4 [==============================] - 3s 861ms/step - loss: 0.8186 - accuracy: 0.6925 - val_loss: 0.6292 - val_accuracy: 0.7721
Epoch 6/20
4/4 [==============================] - 3s 866ms/step - loss: 0.7565 - accuracy: 0.6683 - val_loss: 0.6635 - val_accuracy: 0.7675
Epoch 7/20
4/4 [==============================] - 3s 881ms/step - loss: 0.7050 - accuracy: 0.7208 - val_loss: 0.6445 - val_accuracy: 0.7767
Epoch 8/20
4/4 [==

In [147]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
5/5 [==============================] - 5s 814ms/step - loss: 1.3157 - accuracy: 0.5781 - val_loss: 0.9073 - val_accuracy: 0.6879
Epoch 2/20
5/5 [==============================] - 3s 728ms/step - loss: 0.9604 - accuracy: 0.6340 - val_loss: 0.6694 - val_accuracy: 0.7618
Epoch 3/20
5/5 [==============================] - 3s 738ms/step - loss: 0.8755 - accuracy: 0.6474 - val_loss: 0.6476 - val_accuracy: 0.7668
Epoch 4/20
5/5 [==============================] - 3s 736ms/step - loss: 0.7907 - accuracy: 0.7075 - val_loss: 0.6759 - val_accuracy: 0.7557
Epoch 5/20
5/5 [==============================] - 3s 734ms/step - loss: 0.9041 - accuracy: 0.6749 - val_loss: 0.7049 - val_accuracy: 0.7468
Epoch 6/20
5/5 [==============================] - 3s 728ms/step - loss: 0.7275 - accuracy: 0.7168 - val_loss: 0.6892 - val_accuracy: 0.7500
Epoch 7/20
5/5 [==============================] - 3s 725ms/step - loss: 0.6708 - accuracy: 0.7523 - val_loss: 0.7327 - val_accuracy: 0.7475
Epoch 8/20
5/5 [==

In [148]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
6/6 [==============================] - 5s 743ms/step - loss: 1.5050 - accuracy: 0.4832 - val_loss: 0.7402 - val_accuracy: 0.7678
Epoch 2/20
6/6 [==============================] - 4s 662ms/step - loss: 0.9254 - accuracy: 0.6497 - val_loss: 0.7145 - val_accuracy: 0.7425
Epoch 3/20
6/6 [==============================] - 4s 660ms/step - loss: 0.8913 - accuracy: 0.6140 - val_loss: 0.6438 - val_accuracy: 0.7703
Epoch 4/20
6/6 [==============================] - 4s 674ms/step - loss: 0.7928 - accuracy: 0.6842 - val_loss: 0.7030 - val_accuracy: 0.7703
Epoch 5/20
6/6 [==============================] - 4s 649ms/step - loss: 0.7961 - accuracy: 0.6801 - val_loss: 0.6719 - val_accuracy: 0.7660
Epoch 6/20
6/6 [==============================] - 4s 655ms/step - loss: 0.7476 - accuracy: 0.7309 - val_loss: 0.7375 - val_accuracy: 0.7411
Epoch 7/20
6/6 [==============================] - 4s 661ms/step - loss: 0.7251 - accuracy: 0.7163 - val_loss: 0.7798 - val_accuracy: 0.7290
Epoch 8/20
6/6 [==

In [149]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
8/8 [==============================] - 6s 578ms/step - loss: 1.3273 - accuracy: 0.5363 - val_loss: 0.7760 - val_accuracy: 0.7268
Epoch 2/20
8/8 [==============================] - 4s 545ms/step - loss: 0.9486 - accuracy: 0.6232 - val_loss: 0.9531 - val_accuracy: 0.6109
Epoch 3/20
8/8 [==============================] - 4s 529ms/step - loss: 1.0553 - accuracy: 0.6166 - val_loss: 0.9067 - val_accuracy: 0.6184
Epoch 4/20
8/8 [==============================] - 4s 548ms/step - loss: 0.8559 - accuracy: 0.6546 - val_loss: 1.0737 - val_accuracy: 0.5407
Epoch 5/20
8/8 [==============================] - 4s 530ms/step - loss: 1.0641 - accuracy: 0.5810 - val_loss: 0.7331 - val_accuracy: 0.7254
Epoch 6/20
8/8 [==============================] - 4s 534ms/step - loss: 0.7536 - accuracy: 0.6914 - val_loss: 1.1190 - val_accuracy: 0.5499
Epoch 7/20
8/8 [==============================] - 4s 529ms/step - loss: 0.9336 - accuracy: 0.6521 - val_loss: 0.8824 - val_accuracy: 0.6904
Epoch 8/20
8/8 [==

In [150]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
9/9 [==============================] - 6s 561ms/step - loss: 1.9069 - accuracy: 0.4958 - val_loss: 0.6722 - val_accuracy: 0.7732
Epoch 2/20
9/9 [==============================] - 4s 515ms/step - loss: 0.9456 - accuracy: 0.6357 - val_loss: 0.8047 - val_accuracy: 0.7175
Epoch 3/20
9/9 [==============================] - 4s 519ms/step - loss: 0.9472 - accuracy: 0.6346 - val_loss: 0.8635 - val_accuracy: 0.6819
Epoch 4/20
9/9 [==============================] - 4s 510ms/step - loss: 0.9363 - accuracy: 0.6551 - val_loss: 0.7011 - val_accuracy: 0.7447
Epoch 5/20
9/9 [==============================] - 4s 515ms/step - loss: 0.8889 - accuracy: 0.6627 - val_loss: 0.7562 - val_accuracy: 0.7347
Epoch 6/20
9/9 [==============================] - 4s 512ms/step - loss: 0.8726 - accuracy: 0.6510 - val_loss: 0.6358 - val_accuracy: 0.7778
Epoch 7/20
9/9 [==============================] - 5s 522ms/step - loss: 0.7659 - accuracy: 0.7027 - val_loss: 0.6670 - val_accuracy: 0.7611
Epoch 8/20
9/9 [==

In [151]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
10/10 [==============================] - 7s 533ms/step - loss: 1.2359 - accuracy: 0.5692 - val_loss: 0.6317 - val_accuracy: 0.7789
Epoch 2/20
10/10 [==============================] - 5s 487ms/step - loss: 0.9109 - accuracy: 0.6464 - val_loss: 0.7067 - val_accuracy: 0.7507
Epoch 3/20
10/10 [==============================] - 5s 488ms/step - loss: 0.8669 - accuracy: 0.6676 - val_loss: 0.6474 - val_accuracy: 0.7885
Epoch 4/20
10/10 [==============================] - 5s 487ms/step - loss: 0.7987 - accuracy: 0.6863 - val_loss: 0.6214 - val_accuracy: 0.7860
Epoch 5/20
10/10 [==============================] - 5s 487ms/step - loss: 0.7191 - accuracy: 0.7043 - val_loss: 0.6997 - val_accuracy: 0.7564
Epoch 6/20
10/10 [==============================] - 5s 487ms/step - loss: 0.8263 - accuracy: 0.6649 - val_loss: 0.6521 - val_accuracy: 0.7682
Epoch 7/20
10/10 [==============================] - 5s 490ms/step - loss: 0.7458 - accuracy: 0.7106 - val_loss: 0.6866 - val_accuracy: 0.7546
Epoc

In [152]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
11/11 [==============================] - 7s 518ms/step - loss: 1.4060 - accuracy: 0.4978 - val_loss: 0.6437 - val_accuracy: 0.7989
Epoch 2/20
11/11 [==============================] - 5s 472ms/step - loss: 0.9550 - accuracy: 0.6209 - val_loss: 0.7166 - val_accuracy: 0.7743
Epoch 3/20
11/11 [==============================] - 5s 473ms/step - loss: 0.9084 - accuracy: 0.6365 - val_loss: 0.6045 - val_accuracy: 0.7921
Epoch 4/20
11/11 [==============================] - 5s 471ms/step - loss: 0.8253 - accuracy: 0.6697 - val_loss: 0.6223 - val_accuracy: 0.7892
Epoch 5/20
11/11 [==============================] - 5s 478ms/step - loss: 0.8097 - accuracy: 0.6899 - val_loss: 0.6499 - val_accuracy: 0.7735
Epoch 6/20
11/11 [==============================] - 5s 475ms/step - loss: 0.7665 - accuracy: 0.7106 - val_loss: 0.6948 - val_accuracy: 0.7785
Epoch 7/20
11/11 [==============================] - 5s 475ms/step - loss: 0.6901 - accuracy: 0.7498 - val_loss: 0.6780 - val_accuracy: 0.7603
Epoc

In [153]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
12/12 [==============================] - 7s 501ms/step - loss: 1.3383 - accuracy: 0.5402 - val_loss: 0.6836 - val_accuracy: 0.7500
Epoch 2/20
12/12 [==============================] - 5s 464ms/step - loss: 0.9017 - accuracy: 0.6638 - val_loss: 0.7648 - val_accuracy: 0.7439
Epoch 3/20
12/12 [==============================] - 5s 463ms/step - loss: 0.8931 - accuracy: 0.6704 - val_loss: 0.6182 - val_accuracy: 0.7753
Epoch 4/20
12/12 [==============================] - 6s 468ms/step - loss: 0.8396 - accuracy: 0.6881 - val_loss: 0.7628 - val_accuracy: 0.7404
Epoch 5/20
12/12 [==============================] - 5s 462ms/step - loss: 0.8177 - accuracy: 0.6765 - val_loss: 0.6844 - val_accuracy: 0.7511
Epoch 6/20
12/12 [==============================] - 5s 466ms/step - loss: 0.7785 - accuracy: 0.6921 - val_loss: 0.6915 - val_accuracy: 0.7475
Epoch 7/20
12/12 [==============================] - 5s 465ms/step - loss: 0.7511 - accuracy: 0.7233 - val_loss: 0.6161 - val_accuracy: 0.7878
Epoc

In [154]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
13/13 [==============================] - 8s 499ms/step - loss: 1.2065 - accuracy: 0.5726 - val_loss: 0.7045 - val_accuracy: 0.7482
Epoch 2/20
13/13 [==============================] - 6s 462ms/step - loss: 0.8869 - accuracy: 0.6558 - val_loss: 0.6218 - val_accuracy: 0.7789
Epoch 3/20
13/13 [==============================] - 6s 469ms/step - loss: 0.8741 - accuracy: 0.6588 - val_loss: 0.6239 - val_accuracy: 0.7842
Epoch 4/20
13/13 [==============================] - 6s 465ms/step - loss: 0.8109 - accuracy: 0.6890 - val_loss: 0.6440 - val_accuracy: 0.7917
Epoch 5/20
13/13 [==============================] - 6s 464ms/step - loss: 0.7902 - accuracy: 0.7112 - val_loss: 0.7104 - val_accuracy: 0.7643
Epoch 6/20
13/13 [==============================] - 6s 464ms/step - loss: 0.7276 - accuracy: 0.7094 - val_loss: 0.6719 - val_accuracy: 0.7735
Epoch 7/20
13/13 [==============================] - 6s 465ms/step - loss: 0.7408 - accuracy: 0.7110 - val_loss: 0.6792 - val_accuracy: 0.7685
Epo

In [155]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
15/15 [==============================] - 8s 458ms/step - loss: 1.4541 - accuracy: 0.5139 - val_loss: 0.7720 - val_accuracy: 0.7200
Epoch 2/20
15/15 [==============================] - 6s 430ms/step - loss: 1.0026 - accuracy: 0.6209 - val_loss: 0.6983 - val_accuracy: 0.7614
Epoch 3/20
15/15 [==============================] - 6s 431ms/step - loss: 0.8961 - accuracy: 0.6383 - val_loss: 0.7418 - val_accuracy: 0.7468
Epoch 4/20
15/15 [==============================] - 6s 423ms/step - loss: 0.8662 - accuracy: 0.6632 - val_loss: 0.8753 - val_accuracy: 0.6758
Epoch 5/20
15/15 [==============================] - 6s 426ms/step - loss: 0.8836 - accuracy: 0.6646 - val_loss: 0.7009 - val_accuracy: 0.7414
Epoch 6/20
15/15 [==============================] - 6s 426ms/step - loss: 0.7980 - accuracy: 0.6925 - val_loss: 1.0416 - val_accuracy: 0.6195
Epoch 7/20
15/15 [==============================] - 6s 424ms/step - loss: 0.8307 - accuracy: 0.6864 - val_loss: 0.7516 - val_accuracy: 0.7529
Epo

In [156]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
16/16 [==============================] - 9s 452ms/step - loss: 1.2926 - accuracy: 0.5620 - val_loss: 0.7316 - val_accuracy: 0.7603
Epoch 2/20
16/16 [==============================] - 7s 429ms/step - loss: 0.9059 - accuracy: 0.6518 - val_loss: 0.7063 - val_accuracy: 0.7407
Epoch 3/20
16/16 [==============================] - 7s 433ms/step - loss: 0.9274 - accuracy: 0.6315 - val_loss: 0.7147 - val_accuracy: 0.7364
Epoch 4/20
16/16 [==============================] - 7s 426ms/step - loss: 0.8923 - accuracy: 0.6434 - val_loss: 0.5947 - val_accuracy: 0.7917
Epoch 5/20
16/16 [==============================] - 7s 431ms/step - loss: 0.8125 - accuracy: 0.6888 - val_loss: 0.7624 - val_accuracy: 0.7126
Epoch 6/20
16/16 [==============================] - 7s 428ms/step - loss: 0.7904 - accuracy: 0.6746 - val_loss: 0.7019 - val_accuracy: 0.7564
Epoch 7/20
16/16 [==============================] - 7s 427ms/step - loss: 0.7440 - accuracy: 0.7059 - val_loss: 0.7138 - val_accuracy: 0.7543
Epo

In [157]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
17/17 [==============================] - 9s 458ms/step - loss: 1.5271 - accuracy: 0.5055 - val_loss: 0.6884 - val_accuracy: 0.7660
Epoch 2/20
17/17 [==============================] - 7s 435ms/step - loss: 0.8831 - accuracy: 0.6628 - val_loss: 0.6782 - val_accuracy: 0.7675
Epoch 3/20
17/17 [==============================] - 7s 435ms/step - loss: 0.8632 - accuracy: 0.6670 - val_loss: 0.7030 - val_accuracy: 0.7507
Epoch 4/20
17/17 [==============================] - 7s 437ms/step - loss: 0.8411 - accuracy: 0.6906 - val_loss: 0.6447 - val_accuracy: 0.7850
Epoch 5/20
17/17 [==============================] - 7s 434ms/step - loss: 0.7891 - accuracy: 0.6918 - val_loss: 0.6550 - val_accuracy: 0.7860
Epoch 6/20
17/17 [==============================] - 7s 435ms/step - loss: 0.8095 - accuracy: 0.6879 - val_loss: 0.6551 - val_accuracy: 0.7721
Epoch 7/20
17/17 [==============================] - 7s 444ms/step - loss: 0.7117 - accuracy: 0.7249 - val_loss: 0.6224 - val_accuracy: 0.7828
Epo

In [158]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
18/18 [==============================] - 10s 459ms/step - loss: 1.4190 - accuracy: 0.5130 - val_loss: 0.7417 - val_accuracy: 0.7682
Epoch 2/20
18/18 [==============================] - 8s 436ms/step - loss: 0.9416 - accuracy: 0.6374 - val_loss: 0.6576 - val_accuracy: 0.7696
Epoch 3/20
18/18 [==============================] - 8s 438ms/step - loss: 0.9165 - accuracy: 0.6530 - val_loss: 0.7059 - val_accuracy: 0.7518
Epoch 4/20
18/18 [==============================] - 8s 438ms/step - loss: 0.8507 - accuracy: 0.6680 - val_loss: 0.6225 - val_accuracy: 0.7850
Epoch 5/20
18/18 [==============================] - 8s 436ms/step - loss: 0.8144 - accuracy: 0.6871 - val_loss: 0.7213 - val_accuracy: 0.7489
Epoch 6/20
18/18 [==============================] - 8s 437ms/step - loss: 0.8322 - accuracy: 0.6859 - val_loss: 0.6739 - val_accuracy: 0.7653
Epoch 7/20
18/18 [==============================] - 8s 436ms/step - loss: 0.7143 - accuracy: 0.7202 - val_loss: 0.7622 - val_accuracy: 0.7375
Ep

In [159]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
19/19 [==============================] - 10s 461ms/step - loss: 1.2435 - accuracy: 0.5594 - val_loss: 0.6524 - val_accuracy: 0.7639
Epoch 2/20
19/19 [==============================] - 8s 443ms/step - loss: 0.9133 - accuracy: 0.6580 - val_loss: 0.7021 - val_accuracy: 0.7564
Epoch 3/20
19/19 [==============================] - 8s 444ms/step - loss: 0.8481 - accuracy: 0.6596 - val_loss: 0.7755 - val_accuracy: 0.7097
Epoch 4/20
19/19 [==============================] - 8s 447ms/step - loss: 0.8739 - accuracy: 0.6653 - val_loss: 0.6572 - val_accuracy: 0.7778
Epoch 5/20
19/19 [==============================] - 8s 440ms/step - loss: 0.7692 - accuracy: 0.6939 - val_loss: 0.6664 - val_accuracy: 0.7696
Epoch 6/20
19/19 [==============================] - 8s 441ms/step - loss: 0.7251 - accuracy: 0.7211 - val_loss: 0.6606 - val_accuracy: 0.7675
Epoch 7/20
19/19 [==============================] - 8s 440ms/step - loss: 0.7424 - accuracy: 0.7000 - val_loss: 0.7536 - val_accuracy: 0.7297
Ep

In [160]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
20/20 [==============================] - 11s 463ms/step - loss: 1.2728 - accuracy: 0.5455 - val_loss: 0.7016 - val_accuracy: 0.7386
Epoch 2/20
20/20 [==============================] - 9s 437ms/step - loss: 0.9035 - accuracy: 0.6408 - val_loss: 0.6332 - val_accuracy: 0.7800
Epoch 3/20
20/20 [==============================] - 9s 437ms/step - loss: 0.8664 - accuracy: 0.6627 - val_loss: 0.7846 - val_accuracy: 0.7068
Epoch 4/20
20/20 [==============================] - 9s 442ms/step - loss: 0.8263 - accuracy: 0.6850 - val_loss: 0.6799 - val_accuracy: 0.7693
Epoch 5/20
20/20 [==============================] - 9s 441ms/step - loss: 0.8166 - accuracy: 0.6808 - val_loss: 0.7114 - val_accuracy: 0.7518
Epoch 6/20
20/20 [==============================] - 9s 439ms/step - loss: 0.7590 - accuracy: 0.7097 - val_loss: 0.6147 - val_accuracy: 0.7928
Epoch 7/20
20/20 [==============================] - 9s 439ms/step - loss: 0.7297 - accuracy: 0.7201 - val_loss: 0.6105 - val_accuracy: 0.7885
Ep

In [161]:
time.sleep(400)

In [162]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
22/22 [==============================] - 8s 299ms/step - loss: 1.2890 - accuracy: 0.5360 - val_loss: 0.6527 - val_accuracy: 0.7743
Epoch 2/20
22/22 [==============================] - 6s 283ms/step - loss: 0.9133 - accuracy: 0.6441 - val_loss: 0.9677 - val_accuracy: 0.6369
Epoch 3/20
22/22 [==============================] - 6s 291ms/step - loss: 0.9596 - accuracy: 0.6251 - val_loss: 0.9660 - val_accuracy: 0.5849
Epoch 4/20
22/22 [==============================] - 6s 296ms/step - loss: 0.8725 - accuracy: 0.6487 - val_loss: 0.7873 - val_accuracy: 0.7282
Epoch 5/20
22/22 [==============================] - 7s 299ms/step - loss: 0.8305 - accuracy: 0.6853 - val_loss: 0.7229 - val_accuracy: 0.7350
Epoch 6/20
22/22 [==============================] - 7s 303ms/step - loss: 0.8594 - accuracy: 0.6731 - val_loss: 0.7972 - val_accuracy: 0.7407
Epoch 7/20
22/22 [==============================] - 7s 332ms/step - loss: 0.8085 - accuracy: 0.6954 - val_loss: 0.7950 - val_accuracy: 0.7161
Epo

In [101]:
time.sleep(400)

In [163]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
23/23 [==============================] - 12s 452ms/step - loss: 1.1223 - accuracy: 0.5836 - val_loss: 0.7623 - val_accuracy: 0.7454
Epoch 2/20
23/23 [==============================] - 10s 426ms/step - loss: 0.9021 - accuracy: 0.6584 - val_loss: 0.6269 - val_accuracy: 0.7821
Epoch 3/20
23/23 [==============================] - 10s 424ms/step - loss: 0.8620 - accuracy: 0.6635 - val_loss: 0.7612 - val_accuracy: 0.7482
Epoch 4/20
23/23 [==============================] - 10s 425ms/step - loss: 0.8432 - accuracy: 0.6784 - val_loss: 0.7861 - val_accuracy: 0.7357
Epoch 5/20
23/23 [==============================] - 10s 425ms/step - loss: 0.8416 - accuracy: 0.6739 - val_loss: 0.6768 - val_accuracy: 0.7725
Epoch 6/20
23/23 [==============================] - 10s 426ms/step - loss: 0.7835 - accuracy: 0.7083 - val_loss: 0.6474 - val_accuracy: 0.7764
Epoch 7/20
23/23 [==============================] - 10s 431ms/step - loss: 0.7542 - accuracy: 0.7090 - val_loss: 0.9209 - val_accuracy: 0.6

In [ ]:
time.sleep(400)

In [164]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
24/24 [==============================] - 12s 461ms/step - loss: 1.2418 - accuracy: 0.5625 - val_loss: 0.6692 - val_accuracy: 0.7796
Epoch 2/20
24/24 [==============================] - 11s 445ms/step - loss: 0.8976 - accuracy: 0.6554 - val_loss: 0.7266 - val_accuracy: 0.7550
Epoch 3/20
24/24 [==============================] - 11s 447ms/step - loss: 0.8788 - accuracy: 0.6725 - val_loss: 0.6980 - val_accuracy: 0.7653
Epoch 4/20
24/24 [==============================] - 11s 446ms/step - loss: 0.8751 - accuracy: 0.6645 - val_loss: 0.6802 - val_accuracy: 0.7671
Epoch 5/20
24/24 [==============================] - 11s 448ms/step - loss: 0.8090 - accuracy: 0.6824 - val_loss: 0.6046 - val_accuracy: 0.7839
Epoch 6/20
24/24 [==============================] - 11s 450ms/step - loss: 0.7775 - accuracy: 0.7027 - val_loss: 0.7469 - val_accuracy: 0.7379
Epoch 7/20
24/24 [==============================] - 11s 448ms/step - loss: 0.7862 - accuracy: 0.6991 - val_loss: 0.6761 - val_accuracy: 0.7

In [165]:
new_model_random_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_random_e2"

i=0

for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_random_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_random_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_random_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_random_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_random_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_random_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_random_e2_6\assets
Model 

In [166]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [167]:
gc.collect()

513963

## Training guided by NC

In [168]:
# NC
nc_values = []
for i in range(1,6):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [169]:
nc_values = nc_values[len(x_train):] 
print(len(nc_values))

3000


In [170]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_adversary_training)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_adversary_training)


In [171]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  150
150
1 :
0  ->  300
300
2 :
0  ->  450
450
3 :
0  ->  600
600
4 :
0  ->  750
750
5 :
0  ->  900
900
6 :
0  ->  1050
1050
7 :
0  ->  1200
1200
8 :
0  ->  1350
1350
9 :
0  ->  1500
1500
10 :
0  ->  1650
1650
11 :
0  ->  1800
1800
12 :
0  ->  1950
1950
13 :
0  ->  2100
2100
14 :
0  ->  2250
2250
15 :
0  ->  2400
2400
16 :
0  ->  2550
2550
17 :
0  ->  2700
2700
18 :
0  ->  2850
2850
19 :
Last
0  ->  3000
3000


In [172]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [173]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
2/2 [==============================] - 4s 3s/step - loss: 1.2998 - accuracy: 0.5968 - val_loss: 1.3126 - val_accuracy: 0.5243
Epoch 2/20
2/2 [==============================] - 3s 2s/step - loss: 1.3116 - accuracy: 0.4985 - val_loss: 0.9413 - val_accuracy: 0.6623
Epoch 3/20
2/2 [==============================] - 3s 2s/step - loss: 1.0712 - accuracy: 0.6313 - val_loss: 1.0294 - val_accuracy: 0.6352
Epoch 4/20
2/2 [==============================] - 3s 2s/step - loss: 0.9651 - accuracy: 0.6499 - val_loss: 0.9636 - val_accuracy: 0.6626
Epoch 5/20
2/2 [==============================] - 3s 2s/step - loss: 0.8769 - accuracy: 0.6561 - val_loss: 0.9034 - val_accuracy: 0.6901
Epoch 6/20
2/2 [==============================] - 3s 2s/step - loss: 0.7125 - accuracy: 0.7300 - val_loss: 0.8290 - val_accuracy: 0.7058
Epoch 7/20
2/2 [==============================] - 3s 2s/step - loss: 0.6914 - accuracy: 0.7433 - val_loss: 1.3401 - val_accuracy: 0.6013
Epoch 8/20
2/2 [=======================

In [174]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
3/3 [==============================] - 5s 2s/step - loss: 1.3454 - accuracy: 0.5804 - val_loss: 2.3504 - val_accuracy: 0.3727
Epoch 2/20
3/3 [==============================] - 3s 1s/step - loss: 1.6798 - accuracy: 0.4869 - val_loss: 1.2000 - val_accuracy: 0.5681
Epoch 3/20
3/3 [==============================] - 3s 1s/step - loss: 0.9877 - accuracy: 0.6449 - val_loss: 0.8308 - val_accuracy: 0.7282
Epoch 4/20
3/3 [==============================] - 3s 1s/step - loss: 0.7888 - accuracy: 0.7138 - val_loss: 0.7170 - val_accuracy: 0.7543
Epoch 5/20
3/3 [==============================] - 3s 1s/step - loss: 0.7408 - accuracy: 0.7158 - val_loss: 0.8186 - val_accuracy: 0.7175
Epoch 6/20
3/3 [==============================] - 3s 1s/step - loss: 0.7299 - accuracy: 0.7270 - val_loss: 0.8137 - val_accuracy: 0.7307
Epoch 7/20
3/3 [==============================] - 3s 1s/step - loss: 0.6095 - accuracy: 0.7564 - val_loss: 0.6814 - val_accuracy: 0.7500
Epoch 8/20
3/3 [=======================

In [175]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
4/4 [==============================] - 5s 1s/step - loss: 1.4142 - accuracy: 0.5493 - val_loss: 0.9449 - val_accuracy: 0.6915
Epoch 2/20
4/4 [==============================] - 3s 959ms/step - loss: 0.9580 - accuracy: 0.6185 - val_loss: 0.8760 - val_accuracy: 0.6972
Epoch 3/20
4/4 [==============================] - 3s 971ms/step - loss: 0.8271 - accuracy: 0.6931 - val_loss: 0.7055 - val_accuracy: 0.7632
Epoch 4/20
4/4 [==============================] - 3s 950ms/step - loss: 0.7668 - accuracy: 0.7136 - val_loss: 0.7403 - val_accuracy: 0.7532
Epoch 5/20
4/4 [==============================] - 3s 957ms/step - loss: 0.6959 - accuracy: 0.7349 - val_loss: 0.8298 - val_accuracy: 0.7186
Epoch 6/20
4/4 [==============================] - 3s 953ms/step - loss: 0.7454 - accuracy: 0.7080 - val_loss: 0.8147 - val_accuracy: 0.7304
Epoch 7/20
4/4 [==============================] - 3s 953ms/step - loss: 0.6902 - accuracy: 0.7561 - val_loss: 0.7555 - val_accuracy: 0.7404
Epoch 8/20
4/4 [=====

In [176]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
5/5 [==============================] - 5s 928ms/step - loss: 1.2962 - accuracy: 0.5422 - val_loss: 0.8238 - val_accuracy: 0.7072
Epoch 2/20
5/5 [==============================] - 4s 826ms/step - loss: 0.9699 - accuracy: 0.6465 - val_loss: 0.6625 - val_accuracy: 0.7718
Epoch 3/20
5/5 [==============================] - 4s 826ms/step - loss: 0.7842 - accuracy: 0.6981 - val_loss: 0.7004 - val_accuracy: 0.7636
Epoch 4/20
5/5 [==============================] - 4s 834ms/step - loss: 0.7670 - accuracy: 0.7020 - val_loss: 0.9352 - val_accuracy: 0.6887
Epoch 5/20
5/5 [==============================] - 4s 833ms/step - loss: 0.7980 - accuracy: 0.6778 - val_loss: 1.2100 - val_accuracy: 0.6569
Epoch 6/20
5/5 [==============================] - 4s 830ms/step - loss: 0.7724 - accuracy: 0.7152 - val_loss: 0.8566 - val_accuracy: 0.7150
Epoch 7/20
5/5 [==============================] - 4s 825ms/step - loss: 0.6720 - accuracy: 0.7320 - val_loss: 0.7872 - val_accuracy: 0.7293
Epoch 8/20
5/5 [==

In [177]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
6/6 [==============================] - 6s 811ms/step - loss: 1.3480 - accuracy: 0.5432 - val_loss: 0.7508 - val_accuracy: 0.7482
Epoch 2/20
6/6 [==============================] - 4s 727ms/step - loss: 0.8991 - accuracy: 0.6765 - val_loss: 0.6565 - val_accuracy: 0.7743
Epoch 3/20
6/6 [==============================] - 4s 726ms/step - loss: 0.7618 - accuracy: 0.7034 - val_loss: 0.7132 - val_accuracy: 0.7611
Epoch 4/20
6/6 [==============================] - 4s 723ms/step - loss: 0.7353 - accuracy: 0.7301 - val_loss: 0.6393 - val_accuracy: 0.7803
Epoch 5/20
6/6 [==============================] - 4s 719ms/step - loss: 0.7130 - accuracy: 0.7575 - val_loss: 0.7623 - val_accuracy: 0.7036
Epoch 6/20
6/6 [==============================] - 4s 718ms/step - loss: 0.7391 - accuracy: 0.7108 - val_loss: 0.8571 - val_accuracy: 0.7168
Epoch 7/20
6/6 [==============================] - 4s 720ms/step - loss: 0.6386 - accuracy: 0.7505 - val_loss: 0.9412 - val_accuracy: 0.6726
Epoch 8/20
6/6 [==

In [178]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
8/8 [==============================] - 6s 623ms/step - loss: 1.2881 - accuracy: 0.5600 - val_loss: 0.6871 - val_accuracy: 0.7636
Epoch 2/20
8/8 [==============================] - 4s 559ms/step - loss: 0.9277 - accuracy: 0.6498 - val_loss: 1.2725 - val_accuracy: 0.5075
Epoch 3/20
8/8 [==============================] - 4s 559ms/step - loss: 0.9845 - accuracy: 0.6050 - val_loss: 0.8622 - val_accuracy: 0.7300
Epoch 4/20
8/8 [==============================] - 4s 558ms/step - loss: 0.9155 - accuracy: 0.6617 - val_loss: 0.8067 - val_accuracy: 0.7108
Epoch 5/20
8/8 [==============================] - 4s 558ms/step - loss: 0.9047 - accuracy: 0.6630 - val_loss: 0.7256 - val_accuracy: 0.7243
Epoch 6/20
8/8 [==============================] - 4s 563ms/step - loss: 0.8383 - accuracy: 0.6799 - val_loss: 0.9939 - val_accuracy: 0.6295
Epoch 7/20
8/8 [==============================] - 4s 564ms/step - loss: 0.8057 - accuracy: 0.6744 - val_loss: 0.8656 - val_accuracy: 0.6890
Epoch 8/20
8/8 [==

In [179]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
9/9 [==============================] - 6s 580ms/step - loss: 1.2329 - accuracy: 0.5876 - val_loss: 0.8179 - val_accuracy: 0.7250
Epoch 2/20
9/9 [==============================] - 5s 541ms/step - loss: 0.8760 - accuracy: 0.6669 - val_loss: 0.6378 - val_accuracy: 0.7746
Epoch 3/20
9/9 [==============================] - 5s 538ms/step - loss: 0.8519 - accuracy: 0.6952 - val_loss: 0.7115 - val_accuracy: 0.7529
Epoch 4/20
9/9 [==============================] - 5s 542ms/step - loss: 0.8609 - accuracy: 0.6652 - val_loss: 0.6355 - val_accuracy: 0.7821
Epoch 5/20
9/9 [==============================] - 5s 540ms/step - loss: 0.7400 - accuracy: 0.7269 - val_loss: 0.6825 - val_accuracy: 0.7625
Epoch 6/20
9/9 [==============================] - 5s 550ms/step - loss: 0.7181 - accuracy: 0.7225 - val_loss: 0.8124 - val_accuracy: 0.7150
Epoch 7/20
9/9 [==============================] - 5s 541ms/step - loss: 0.7836 - accuracy: 0.7176 - val_loss: 0.7219 - val_accuracy: 0.7404
Epoch 8/20
9/9 [==

In [180]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
10/10 [==============================] - 7s 562ms/step - loss: 1.4000 - accuracy: 0.5530 - val_loss: 0.6661 - val_accuracy: 0.7782
Epoch 2/20
10/10 [==============================] - 5s 524ms/step - loss: 0.8767 - accuracy: 0.6598 - val_loss: 0.7222 - val_accuracy: 0.7436
Epoch 3/20
10/10 [==============================] - 5s 523ms/step - loss: 0.8825 - accuracy: 0.6588 - val_loss: 0.7402 - val_accuracy: 0.7400
Epoch 4/20
10/10 [==============================] - 5s 523ms/step - loss: 0.8414 - accuracy: 0.6754 - val_loss: 0.7487 - val_accuracy: 0.7443
Epoch 5/20
10/10 [==============================] - 5s 522ms/step - loss: 0.7868 - accuracy: 0.6978 - val_loss: 0.6794 - val_accuracy: 0.7593
Epoch 6/20
10/10 [==============================] - 5s 517ms/step - loss: 0.7519 - accuracy: 0.6978 - val_loss: 0.7707 - val_accuracy: 0.7200
Epoch 7/20
10/10 [==============================] - 5s 525ms/step - loss: 0.7603 - accuracy: 0.6925 - val_loss: 0.7569 - val_accuracy: 0.7372
Epoc

In [181]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
11/11 [==============================] - 7s 545ms/step - loss: 1.5189 - accuracy: 0.5131 - val_loss: 0.8777 - val_accuracy: 0.6979
Epoch 2/20
11/11 [==============================] - 6s 514ms/step - loss: 0.9257 - accuracy: 0.6439 - val_loss: 0.7167 - val_accuracy: 0.7618
Epoch 3/20
11/11 [==============================] - 6s 521ms/step - loss: 0.8629 - accuracy: 0.6731 - val_loss: 0.7805 - val_accuracy: 0.7422
Epoch 4/20
11/11 [==============================] - 5s 510ms/step - loss: 0.8529 - accuracy: 0.6791 - val_loss: 0.6642 - val_accuracy: 0.7796
Epoch 5/20
11/11 [==============================] - 5s 509ms/step - loss: 0.8418 - accuracy: 0.6589 - val_loss: 0.6804 - val_accuracy: 0.7646
Epoch 6/20
11/11 [==============================] - 5s 504ms/step - loss: 0.7307 - accuracy: 0.7105 - val_loss: 0.6481 - val_accuracy: 0.7792
Epoch 7/20
11/11 [==============================] - 5s 504ms/step - loss: 0.7552 - accuracy: 0.6897 - val_loss: 0.6599 - val_accuracy: 0.7757
Epoc

In [182]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
12/12 [==============================] - 8s 534ms/step - loss: 1.2468 - accuracy: 0.5556 - val_loss: 0.6503 - val_accuracy: 0.7789
Epoch 2/20
12/12 [==============================] - 6s 505ms/step - loss: 0.9232 - accuracy: 0.6414 - val_loss: 0.6957 - val_accuracy: 0.7564
Epoch 3/20
12/12 [==============================] - 6s 501ms/step - loss: 0.9074 - accuracy: 0.6409 - val_loss: 0.7140 - val_accuracy: 0.7550
Epoch 4/20
12/12 [==============================] - 6s 505ms/step - loss: 0.8300 - accuracy: 0.6817 - val_loss: 0.6718 - val_accuracy: 0.7718
Epoch 5/20
12/12 [==============================] - 6s 501ms/step - loss: 0.8155 - accuracy: 0.7022 - val_loss: 0.9849 - val_accuracy: 0.6430
Epoch 6/20
12/12 [==============================] - 6s 503ms/step - loss: 0.8495 - accuracy: 0.6709 - val_loss: 0.6714 - val_accuracy: 0.7678
Epoch 7/20
12/12 [==============================] - 6s 499ms/step - loss: 0.7722 - accuracy: 0.7091 - val_loss: 0.7238 - val_accuracy: 0.7529
Epoc

In [183]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
13/13 [==============================] - 8s 520ms/step - loss: 1.3417 - accuracy: 0.5112 - val_loss: 0.6316 - val_accuracy: 0.7782
Epoch 2/20
13/13 [==============================] - 7s 519ms/step - loss: 0.9269 - accuracy: 0.6462 - val_loss: 0.6811 - val_accuracy: 0.7650
Epoch 3/20
13/13 [==============================] - 6s 492ms/step - loss: 0.9696 - accuracy: 0.6126 - val_loss: 0.6939 - val_accuracy: 0.7625
Epoch 4/20
13/13 [==============================] - 6s 492ms/step - loss: 0.8321 - accuracy: 0.6640 - val_loss: 0.6717 - val_accuracy: 0.7743
Epoch 5/20
13/13 [==============================] - 6s 491ms/step - loss: 0.7713 - accuracy: 0.6811 - val_loss: 0.9193 - val_accuracy: 0.7004
Epoch 6/20
13/13 [==============================] - 6s 493ms/step - loss: 0.8136 - accuracy: 0.6798 - val_loss: 0.7659 - val_accuracy: 0.7297
Epoch 7/20
13/13 [==============================] - 6s 493ms/step - loss: 0.7636 - accuracy: 0.6842 - val_loss: 0.6926 - val_accuracy: 0.7714
Epo

In [184]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
15/15 [==============================] - 9s 483ms/step - loss: 1.5287 - accuracy: 0.4993 - val_loss: 0.7754 - val_accuracy: 0.7275
Epoch 2/20
15/15 [==============================] - 7s 451ms/step - loss: 0.9695 - accuracy: 0.6228 - val_loss: 0.8690 - val_accuracy: 0.7061
Epoch 3/20
15/15 [==============================] - 7s 449ms/step - loss: 0.9966 - accuracy: 0.6410 - val_loss: 0.7653 - val_accuracy: 0.7279
Epoch 4/20
15/15 [==============================] - 7s 447ms/step - loss: 0.8918 - accuracy: 0.6613 - val_loss: 0.7635 - val_accuracy: 0.7115
Epoch 5/20
15/15 [==============================] - 7s 446ms/step - loss: 0.8552 - accuracy: 0.6754 - val_loss: 0.8624 - val_accuracy: 0.6994
Epoch 6/20
15/15 [==============================] - 7s 448ms/step - loss: 0.8683 - accuracy: 0.6584 - val_loss: 0.8592 - val_accuracy: 0.6705
Epoch 7/20
15/15 [==============================] - 7s 446ms/step - loss: 0.8482 - accuracy: 0.6750 - val_loss: 0.7370 - val_accuracy: 0.7436
Epo

In [185]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
16/16 [==============================] - 9s 463ms/step - loss: 1.1698 - accuracy: 0.5880 - val_loss: 0.8691 - val_accuracy: 0.6862
Epoch 2/20
16/16 [==============================] - 7s 433ms/step - loss: 0.9252 - accuracy: 0.6387 - val_loss: 0.6895 - val_accuracy: 0.7600
Epoch 3/20
16/16 [==============================] - 7s 431ms/step - loss: 0.8664 - accuracy: 0.6670 - val_loss: 0.7153 - val_accuracy: 0.7525
Epoch 4/20
16/16 [==============================] - 7s 434ms/step - loss: 0.8418 - accuracy: 0.6744 - val_loss: 0.7629 - val_accuracy: 0.7361
Epoch 5/20
16/16 [==============================] - 7s 431ms/step - loss: 0.8073 - accuracy: 0.6903 - val_loss: 0.7185 - val_accuracy: 0.7596
Epoch 6/20
16/16 [==============================] - 7s 431ms/step - loss: 0.7715 - accuracy: 0.7049 - val_loss: 0.7306 - val_accuracy: 0.7607
Epoch 7/20
16/16 [==============================] - 7s 429ms/step - loss: 0.7854 - accuracy: 0.7155 - val_loss: 0.7948 - val_accuracy: 0.7329
Epo

In [186]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
17/17 [==============================] - 9s 425ms/step - loss: 1.2300 - accuracy: 0.5568 - val_loss: 0.6521 - val_accuracy: 0.7693
Epoch 2/20
17/17 [==============================] - 7s 403ms/step - loss: 0.9933 - accuracy: 0.6229 - val_loss: 0.7396 - val_accuracy: 0.7411
Epoch 3/20
17/17 [==============================] - 7s 404ms/step - loss: 0.8874 - accuracy: 0.6408 - val_loss: 0.6924 - val_accuracy: 0.7625
Epoch 4/20
17/17 [==============================] - 7s 403ms/step - loss: 0.8259 - accuracy: 0.6753 - val_loss: 0.6881 - val_accuracy: 0.7525
Epoch 5/20
17/17 [==============================] - 7s 404ms/step - loss: 0.8354 - accuracy: 0.6692 - val_loss: 0.6912 - val_accuracy: 0.7586
Epoch 6/20
17/17 [==============================] - 7s 409ms/step - loss: 0.7758 - accuracy: 0.7007 - val_loss: 0.7865 - val_accuracy: 0.7222
Epoch 7/20
17/17 [==============================] - 7s 405ms/step - loss: 0.7870 - accuracy: 0.6964 - val_loss: 0.7872 - val_accuracy: 0.7150
Epo

In [187]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
18/18 [==============================] - 9s 420ms/step - loss: 1.2172 - accuracy: 0.5597 - val_loss: 0.7604 - val_accuracy: 0.7422
Epoch 2/20
18/18 [==============================] - 7s 399ms/step - loss: 0.9274 - accuracy: 0.6380 - val_loss: 0.6573 - val_accuracy: 0.7657
Epoch 3/20
18/18 [==============================] - 7s 403ms/step - loss: 0.8510 - accuracy: 0.6743 - val_loss: 0.7282 - val_accuracy: 0.7293
Epoch 4/20
18/18 [==============================] - 7s 402ms/step - loss: 0.8539 - accuracy: 0.6805 - val_loss: 0.7703 - val_accuracy: 0.7343
Epoch 5/20
18/18 [==============================] - 7s 399ms/step - loss: 0.8406 - accuracy: 0.6794 - val_loss: 0.7064 - val_accuracy: 0.7536
Epoch 6/20
18/18 [==============================] - 7s 399ms/step - loss: 0.8257 - accuracy: 0.6872 - val_loss: 0.6420 - val_accuracy: 0.7732
Epoch 7/20
18/18 [==============================] - 7s 398ms/step - loss: 0.7644 - accuracy: 0.7045 - val_loss: 0.7091 - val_accuracy: 0.7571
Epo

In [188]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
19/19 [==============================] - 9s 415ms/step - loss: 1.1453 - accuracy: 0.5833 - val_loss: 0.7563 - val_accuracy: 0.7514
Epoch 2/20
19/19 [==============================] - 7s 398ms/step - loss: 0.8942 - accuracy: 0.6617 - val_loss: 0.7954 - val_accuracy: 0.7393
Epoch 3/20
19/19 [==============================] - 7s 396ms/step - loss: 0.8573 - accuracy: 0.6642 - val_loss: 0.7479 - val_accuracy: 0.7290
Epoch 4/20
19/19 [==============================] - 7s 394ms/step - loss: 0.8113 - accuracy: 0.6792 - val_loss: 0.7076 - val_accuracy: 0.7425
Epoch 5/20
19/19 [==============================] - 7s 393ms/step - loss: 0.8182 - accuracy: 0.6798 - val_loss: 0.7751 - val_accuracy: 0.7183
Epoch 6/20
19/19 [==============================] - 7s 393ms/step - loss: 0.7804 - accuracy: 0.6997 - val_loss: 0.6188 - val_accuracy: 0.7796
Epoch 7/20
19/19 [==============================] - 7s 395ms/step - loss: 0.7418 - accuracy: 0.7015 - val_loss: 0.6223 - val_accuracy: 0.7817
Epo

In [189]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
20/20 [==============================] - 10s 413ms/step - loss: 1.3175 - accuracy: 0.5177 - val_loss: 0.8231 - val_accuracy: 0.6976
Epoch 2/20
20/20 [==============================] - 8s 398ms/step - loss: 0.9124 - accuracy: 0.6462 - val_loss: 0.6088 - val_accuracy: 0.7850
Epoch 3/20
20/20 [==============================] - 8s 398ms/step - loss: 0.8482 - accuracy: 0.6747 - val_loss: 0.6463 - val_accuracy: 0.7775
Epoch 4/20
20/20 [==============================] - 8s 399ms/step - loss: 0.8094 - accuracy: 0.6795 - val_loss: 0.6167 - val_accuracy: 0.7735
Epoch 5/20
20/20 [==============================] - 8s 405ms/step - loss: 0.8001 - accuracy: 0.6882 - val_loss: 0.7659 - val_accuracy: 0.7461
Epoch 6/20
20/20 [==============================] - 8s 402ms/step - loss: 0.8164 - accuracy: 0.6865 - val_loss: 0.6251 - val_accuracy: 0.7767
Epoch 7/20
20/20 [==============================] - 8s 401ms/step - loss: 0.7265 - accuracy: 0.7110 - val_loss: 0.7913 - val_accuracy: 0.7315
Ep

In [190]:
time.sleep(400)

In [191]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
22/22 [==============================] - 7s 254ms/step - loss: 1.2032 - accuracy: 0.5395 - val_loss: 1.2818 - val_accuracy: 0.5549
Epoch 2/20
22/22 [==============================] - 5s 237ms/step - loss: 1.0340 - accuracy: 0.6227 - val_loss: 0.7347 - val_accuracy: 0.7389
Epoch 3/20
22/22 [==============================] - 5s 238ms/step - loss: 0.9110 - accuracy: 0.6581 - val_loss: 0.7340 - val_accuracy: 0.7350
Epoch 4/20
22/22 [==============================] - 5s 241ms/step - loss: 0.8275 - accuracy: 0.6836 - val_loss: 0.7481 - val_accuracy: 0.7379
Epoch 5/20
22/22 [==============================] - 5s 247ms/step - loss: 0.8445 - accuracy: 0.6793 - val_loss: 0.7011 - val_accuracy: 0.7596
Epoch 6/20
22/22 [==============================] - 6s 256ms/step - loss: 0.8011 - accuracy: 0.6832 - val_loss: 0.6708 - val_accuracy: 0.7564
Epoch 7/20
22/22 [==============================] - 6s 257ms/step - loss: 0.7932 - accuracy: 0.6905 - val_loss: 0.8173 - val_accuracy: 0.6951
Epo

In [192]:
time.sleep(400)

In [193]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
23/23 [==============================] - 7s 256ms/step - loss: 1.2524 - accuracy: 0.5560 - val_loss: 0.7098 - val_accuracy: 0.7611
Epoch 2/20
23/23 [==============================] - 6s 241ms/step - loss: 0.9047 - accuracy: 0.6466 - val_loss: 0.8639 - val_accuracy: 0.7225
Epoch 3/20
23/23 [==============================] - 5s 239ms/step - loss: 0.8642 - accuracy: 0.6793 - val_loss: 0.7637 - val_accuracy: 0.7136
Epoch 4/20
23/23 [==============================] - 6s 243ms/step - loss: 0.8450 - accuracy: 0.6791 - val_loss: 0.6562 - val_accuracy: 0.7696
Epoch 5/20
23/23 [==============================] - 6s 249ms/step - loss: 0.8003 - accuracy: 0.6767 - val_loss: 0.8039 - val_accuracy: 0.7019
Epoch 6/20
23/23 [==============================] - 6s 259ms/step - loss: 0.8167 - accuracy: 0.6905 - val_loss: 0.6410 - val_accuracy: 0.7728
Epoch 7/20
23/23 [==============================] - 6s 262ms/step - loss: 0.7678 - accuracy: 0.6961 - val_loss: 0.7230 - val_accuracy: 0.7268
Epo

In [ ]:
time.sleep(400)

In [194]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
24/24 [==============================] - 11s 402ms/step - loss: 1.2046 - accuracy: 0.5812 - val_loss: 0.9033 - val_accuracy: 0.6655
Epoch 2/20
24/24 [==============================] - 9s 386ms/step - loss: 0.9008 - accuracy: 0.6419 - val_loss: 0.7994 - val_accuracy: 0.6879
Epoch 3/20
24/24 [==============================] - 9s 391ms/step - loss: 0.8692 - accuracy: 0.6692 - val_loss: 0.7028 - val_accuracy: 0.7628
Epoch 4/20
24/24 [==============================] - 9s 387ms/step - loss: 0.8307 - accuracy: 0.6773 - val_loss: 0.6364 - val_accuracy: 0.7721
Epoch 5/20
24/24 [==============================] - 9s 387ms/step - loss: 0.7739 - accuracy: 0.7058 - val_loss: 0.7625 - val_accuracy: 0.7193
Epoch 6/20
24/24 [==============================] - 9s 392ms/step - loss: 0.8024 - accuracy: 0.6842 - val_loss: 0.7250 - val_accuracy: 0.7343
Epoch 7/20
24/24 [==============================] - 9s 389ms/step - loss: 0.7871 - accuracy: 0.7067 - val_loss: 0.6316 - val_accuracy: 0.7714
Ep

In [195]:
new_model_nc_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_nc_e2"

i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_nc_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_nc_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_nc_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_nc_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_nc_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_nc_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/intel/C3/e2/intel_model_c3_aug_nc_e2_6\assets
Model has been saved
INFO:tensorfl

In [ ]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [ ]:
gc.collect()

## Evaluating

In [ ]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1